<a href="https://colab.research.google.com/github/htleffew/hiring_analytics_simulator/blob/main/keeper_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
"""
This script processes employee snapshot and history data to generate
organizational insights, statistical summaries, data visualizations,
and narrative reports, culminating in an HTML dashboard.

This first part covers:
- Importing necessary libraries.
- Configuring file paths and settings.
- Defining helper functions for validation and formatting.
- Loading and processing the raw employee data.
- Defining the initial narrative generation function including statistical analysis.
"""

# =============================================================================
# # 1. Imports
# =============================================================================
# Description: This section imports all the necessary Python libraries
# required for data manipulation, visualization, statistical analysis,
# JSON handling, date/time operations, and template rendering.

import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import datetime as dt
import json
import traceback
import numpy as np
from collections import defaultdict
import jinja2
import statsmodels.api as sm
from scipy.stats import ttest_ind, chi2_contingency, pearsonr
import re
import os


# =============================================================================
# # 2. Configuration
# =============================================================================
# Description: This section defines global configuration variables, including
# input file names, output file name, template file name, the report date,
# and color palettes for visualizations. It also sets the default Plotly
# theme to ensure visual consistency with the dashboard's intended style.

# --- File Paths ---
snapshot_file = "employee_snapshot.csv"
history_file = "employee_history.csv"
output_html = "gem_ip_dashboard.html"
template_file = "gem_ip_template.html"

# --- Report Settings ---
report_date = dt.datetime.now() # Use current date/time for the report

# --- Visualization Settings ---
# Define color palette matching the target dashboard theme (e.g., Netflix-inspired)
color_palette = {
    'primary': '#e50914',      # Example: Netflix red
    'secondary': '#f5f5f1',    # Example: Netflix white/light gray
    'tertiary': '#b3b3b3',    # Example: Netflix medium gray
    'dark_gray': '#1f1f1f',    # Example: Netflix dark gray
    'black': '#141414'         # Example: Netflix black
}

# Set default Plotly theme for consistency
# This theme is applied globally but can be overridden by specific chart configurations
# or JavaScript within the HTML template.
pio.templates.default = "plotly_dark"
pio.templates["plotly_dark"].layout.update(
    paper_bgcolor='rgba(0,0,0,0)',      # Transparent background for embedding
    plot_bgcolor='rgba(31,31,31,0.5)', # Semi-transparent dark plot area
    font_color=color_palette['secondary'],
    font_family="Segoe UI, sans-serif" # Specify a clean, professional font
)

# =============================================================================
# # 3. Helper Functions
# =============================================================================
# Description: This section contains utility functions used throughout the
# script for tasks like JSON validation, date formatting, safe numerical
# rounding, and calculating organizational scope (total reports).

def validate_json(json_string, variable_name):
    """
    Validates if a given string is parsable as JSON.

    Prints a warning if the input is not a string or an error if parsing fails.
    This helps ensure that data passed to the HTML template is correctly formatted.

    Args:
        json_string (any): The input to validate. Expected to be a string.
        variable_name (str): The name of the variable being validated (for logging).

    Returns:
        bool: True if the input is a valid JSON string or not a string, False otherwise.
    """
    if not isinstance(json_string, str):
        # If the input isn't a string, bypass JSON validation but log a warning.
        print(f"--- VALIDATION WARNING for {variable_name}: Input is not a string (type: {type(json_string)}). Skipping json.loads validation. ---")
        return True # Assuming non-strings are acceptable if not intended to be JSON
    try:
        # Attempt to parse the string as JSON
        json.loads(json_string)
        # print(f"Validation successful for: {variable_name}") # Optional: Success message for debugging
        return True
    except json.JSONDecodeError as e:
        # Handle specific JSON parsing errors
        print(f"--- INVALID JSON DETECTED in {variable_name} ---")
        print(f"Error message: {e}")
        return False
    except Exception as e:
        # Catch any other unexpected errors during validation
        print(f"--- UNEXPECTED ERROR during validation of {variable_name} ---")
        print(f"Error: {e}")
        return False

def format_timestamp(value):
    """
    Formats various date/time representations into a 'YYYY-MM-DD' string.

    Handles pandas Timestamps, NaT (Not a Time), and string inputs. Attempts
    to parse strings using common formats before falling back. Returns None
    for null or empty values.

    Args:
        value (any): The date/time value to format (Timestamp, NaT, str, etc.).

    Returns:
        str or None: The formatted date string ('YYYY-MM-DD'), the original
                     string if unparseable (and not empty), or None for null/empty inputs.
    """
    if pd.isnull(value):
        return None
    if isinstance(value, str):
        if not value or value.lower() == 'nan': # Check for empty or 'nan' strings
             return None
        try:
            # Attempt parsing with a specific format first (e.g., YYYYMMDD)
            return pd.to_datetime(value, format='%Y%m%d').strftime('%Y-%m-%d')
        except ValueError:
            try:
                 # Attempt generic parsing if the specific format fails
                 return pd.to_datetime(value).strftime('%Y-%m-%d')
            except ValueError:
                 # If all parsing fails, return the original non-empty string
                 return value
    elif isinstance(value, pd.Timestamp):
        # Format pandas Timestamp objects
        return value.strftime('%Y-%m-%d')
    else:
        # Fallback for other types (e.g., integers representing dates)
        return str(value)

def safe_round(value, decimals=1, na_value='N/A'):
    """
    Safely rounds a numerical value to a specified number of decimal places.

    Handles non-numeric inputs (like NaN, None, or strings) by returning a
    specified placeholder value ('N/A' by default).

    Args:
        value (any): The value to round.
        decimals (int): The number of decimal places to round to. Defaults to 1.
        na_value (str): The value to return if the input is not a valid number.
                        Defaults to 'N/A'.

    Returns:
        float or str: The rounded number, or the na_value placeholder.
    """
    if pd.isna(value) or not isinstance(value, (int, float, np.number)):
        return na_value
    try:
        # Use np.round for compatibility with numpy types
        return np.round(value, decimals)
    except (TypeError, ValueError):
         # Catch potential errors during rounding itself
         return na_value

def calculate_total_scope(employee_id, manager_map, visited):
    """
    Recursively calculates the total organizational scope for a manager.

    Scope includes both direct and indirect reports down the hierarchy.
    Uses a set to avoid double-counting and a visited set to prevent infinite
    loops in case of circular reporting structures.

    Args:
        employee_id (int): The Employee ID of the manager whose scope is being calculated.
        manager_map (dict): A dictionary where keys are manager IDs and values are lists
                            of their direct report employee IDs.
        visited (set): A set containing employee IDs already visited in the current
                       recursion path to detect cycles.

    Returns:
        set: A set containing the Employee IDs of all direct and indirect reports.
    """
    # Base case: If we've already processed this employee in this path, stop.
    if employee_id in visited:
        return set()
    visited.add(employee_id)

    # Get direct reports for the current employee
    direct_reports = manager_map.get(employee_id, [])
    total_reports_set = set(direct_reports) # Start with direct reports

    # Recursively find reports of reports
    for report_id in direct_reports:
        # Pass a copy of visited to avoid interference between sibling branches
        indirect_reports = calculate_total_scope(report_id, manager_map, visited.copy())
        total_reports_set.update(indirect_reports) # Add indirect reports to the set

    return total_reports_set

# =============================================================================
# # 4. Data Loading and Processing
# =============================================================================
# Description: This section defines the `load_data` function, which handles
# reading the employee snapshot and history CSV files, merging them, cleaning
# data types, standardizing values (like job functions and levels), deriving
# new features (like tenure, role, span of control, total scope), and
# identifying data quality issues (like invalid dates or illogical promotion timing).

def load_data():
    """
    Loads, merges, cleans, and processes employee data from CSV files.

    Reads 'employee_snapshot.csv' and 'employee_history.csv', merges them on
    'employee_id', standardizes column names, cleans job functions and levels,
    determines employee roles (Manager/IC), processes dates, calculates derived
    metrics like tenure, time-to-promotion, span of control, and total scope.
    It also identifies and separates records with data quality issues.

    Returns:
        tuple: Contains:
            - df_clean (pd.DataFrame): The main processed DataFrame with valid data.
            - bad_dates_json (str): JSON string listing records with date format issues.
            - promo_issue_json (str): JSON string listing records where promotion date
                                      is illogical (e.g., before hire date).
            Returns (None, "[]", "[]") if file loading or critical processing fails.
    """
    try:
        print(f"Loading data from {snapshot_file} and {history_file}...")
        # Read CSVs, explicitly setting dtype to string initially to avoid type inference issues
        snap = pd.read_csv(snapshot_file, dtype=str)
        hist = pd.read_csv(history_file, dtype=str)

        # --- Column Name Standardization ---
        snap.columns = snap.columns.str.lower().str.replace(' ', '_')
        hist.columns = hist.columns.str.lower().str.replace(' ', '_')

        # --- Data Validation: Check for Required Columns ---
        required_snap_cols = ['employee_id', 'mgr_employee_id', 'job_level', 'job_function']
        required_hist_cols = ['employee_id', 'hire_date', 'promotion_date']

        missing_snap = [col for col in required_snap_cols if col not in snap.columns]
        missing_hist = [col for col in required_hist_cols if col not in hist.columns]

        if missing_snap:
             raise ValueError(f"Snapshot CSV missing required columns: {missing_snap}. Found: {snap.columns.tolist()}")
        if missing_hist:
             raise ValueError(f"History CSV missing required columns: {missing_hist}. Found: {hist.columns.tolist()}")

        # --- Data Type Conversion and Cleaning: IDs ---
        # Convert IDs to numeric, coercing errors to NaN for later removal
        snap['employee_id'] = pd.to_numeric(snap['employee_id'], errors='coerce')
        hist['employee_id'] = pd.to_numeric(hist['employee_id'], errors='coerce')
        snap['mgr_employee_id'] = pd.to_numeric(snap['mgr_employee_id'], errors='coerce')

        # Drop rows where essential IDs couldn't be converted to numeric
        snap.dropna(subset=['employee_id'], inplace=True)
        hist.dropna(subset=['employee_id'], inplace=True)

        # Convert valid employee IDs to integers
        snap['employee_id'] = snap['employee_id'].astype(int)
        hist['employee_id'] = hist['employee_id'].astype(int)

        # --- Merge DataFrames ---
        # Perform a left merge to keep all snapshot records and add history info where available
        df = pd.merge(snap, hist, on='employee_id', how='left')

        # --- Clean Job Function ---
        # Consolidate variations and fill missing values
        df['job_function'] = df['job_function'].replace({
            'Product': 'Product Development',
            'Content': 'Content and Studio',
            'Studio': 'Content and Studio'
        }).fillna('Other') # Fill NaNs with 'Other'
        # Standardize functions outside the main focus groups into 'Other'
        valid_functions = ['Product Development', 'Content and Studio']
        df['job_function'] = df['job_function'].apply(lambda x: x if x in valid_functions else 'Other')

        # --- Clean Job Level ---
        # Store original level string, handle NaNs
        df['job_level_raw'] = df['job_level'].astype(str).str.strip().fillna('Unknown')
        # Extract numeric part (level number) if present at the beginning
        df['specific_level_num'] = df['job_level_raw'].str.extract(r'^(\d+)').iloc[:, 0].astype('Int64') # Use Int64 for nullable integer
        # Extract title part, removing number and separators
        df['job_level_title'] = df['job_level_raw'].str.replace(r'^\d+\s*-\s*', '', regex=True) # Remove "## - "
        df['job_level_title'] = df['job_level_title'].str.replace(r'^\d+\s*', '', regex=True)      # Remove "## "
        df['job_level_title'] = df['job_level_title'].replace('', 'Unknown') # Handle empty strings after replacements
        # Standardize various representations of unknown/missing levels
        df.loc[df['job_level_title'].isin(['nan', 'None', 'unknown']), 'job_level_title'] = 'Unknown'

        # --- Determine Role (Manager/IC) ---
        # Define keywords indicating a management role
        manager_titles = ['Manager', 'Director', 'Vice President', 'VP']
        # Initial role assignment based on job title keywords (case-insensitive)
        df['role'] = df['job_level_title'].apply(
            lambda x: 'People Manager' if any(title.lower() in str(x).lower() for title in manager_titles) else 'Individual Contributor'
        )
        # Refine role based on reporting structure: if someone is listed as a manager for others, mark them as manager
        # Ensure mgr_employee_id is treated as numeric before creating the set
        valid_mgr_ids = pd.to_numeric(df['mgr_employee_id'], errors='coerce').dropna()
        mgr_ids_with_reports = set(valid_mgr_ids.astype(int))
        df.loc[df['employee_id'].isin(mgr_ids_with_reports), 'role'] = 'People Manager'
        df['is_manager'] = (df['role'] == 'People Manager') # Create a boolean flag for convenience

        # --- Process Dates ---
        # Store original date strings for diagnostics
        df['hire_date_str'] = df['hire_date'].astype(str).str.strip()
        df['promotion_date_str'] = df['promotion_date'].astype(str).str.strip()

        # Convert date strings to datetime objects, coercing errors to NaT (Not a Time)
        # Assuming YYYYMMDD format based on original code
        df['hire_date_dt'] = pd.to_datetime(df['hire_date_str'], format='%Y%m%d', errors='coerce')
        df['promotion_date_dt'] = pd.to_datetime(df['promotion_date_str'], format='%Y%m%d', errors='coerce')

        # --- Calculate Derived Fields ---
        # Extract year and quarter from hire date
        df['hire_year'] = df['hire_date_dt'].dt.year.astype('Int64')
        df['hire_quarter_num'] = df['hire_date_dt'].dt.quarter.astype('Int64')
        # Create a string representation of the hire quarter (e.g., '2023Q1')
        df['hire_quarter'] = df['hire_date_dt'].dt.to_period('Q').astype(str).replace('NaT', pd.NA)

        # Extract promotion year
        df['promotion_year'] = df['promotion_date_dt'].dt.year.astype('Int64')

        # Calculate tenure in years (as a float)
        df['tenure_years'] = ((report_date - df['hire_date_dt']).dt.days / 365.25)

        # Calculate time to promotion in years (as a float)
        df['time_to_promotion'] = ((df['promotion_date_dt'] - df['hire_date_dt']).dt.days / 365.25)
        # Invalidate TTP if the promotion date is illogical (before hire date)
        df.loc[df['time_to_promotion'] < 0, 'time_to_promotion'] = pd.NA

        # Create a boolean flag indicating if an employee has a valid promotion record
        df['is_promoted'] = df['promotion_date_dt'].notna() & (df['time_to_promotion'].notna()) # Requires valid date and non-negative TTP

        # --- Calculate Span of Control (Direct Reports) ---
        # Count occurrences of each manager ID in the 'mgr_employee_id' column
        # Ensure mgr_employee_id is numeric before value_counts
        valid_mgr_ids_span = pd.to_numeric(df['mgr_employee_id'], errors='coerce').dropna()
        direct_span = valid_mgr_ids_span.astype(int).value_counts()
        # Map the counts to each employee; fill with 0 for non-managers or those with no reports
        df['span_of_control'] = df['employee_id'].map(direct_span).fillna(0).astype(int)

        # --- Calculate Total Scope (Direct + Indirect Reports) ---
        # Build a map of manager -> list of direct reports
        manager_map = defaultdict(list)
        map_df = df[['employee_id', 'mgr_employee_id']].dropna() # Use only rows with valid IDs
        for _, row in map_df.iterrows():
             try:
                 mgr_id = int(row['mgr_employee_id'])
                 emp_id = int(row['employee_id'])
                 # Avoid self-references which can break recursive calculations
                 if mgr_id != emp_id:
                    manager_map[mgr_id].append(emp_id)
             except (ValueError, TypeError):
                 # Silently ignore rows where IDs are not valid integers
                 pass

        # Calculate scope for each manager using the recursive helper function
        print("Calculating organizational scope for managers...")
        scope_map = {}
        manager_employee_ids = df.loc[df['is_manager'], 'employee_id'].dropna().unique()
        for mid in manager_employee_ids:
             try:
                 mid_int = int(mid) # Ensure ID is integer
                 # The calculate_total_scope function returns a set of all reports
                 scope_map[mid_int] = len(calculate_total_scope(mid_int, manager_map, set()))
             except (ValueError, TypeError):
                 # Silently ignore invalid manager IDs
                 pass
        # Map the calculated scope to each employee; fill with 0 for non-managers
        df['total_scope'] = df['employee_id'].map(scope_map).fillna(0).astype(int)

        # --- Identify Data Quality Issues ---
        print("Identifying data quality issues...")
        # Find records where original date strings exist but failed parsing
        bad_hire = df.loc[df['hire_date_dt'].isna() & df['hire_date_str'].notna() & (df['hire_date_str'] != 'nan') & (df['hire_date_str'] != ''),
                          ['employee_id', 'hire_date_str']]
        bad_promo = df.loc[df['promotion_date_dt'].isna() & df['promotion_date_str'].notna() & (df['promotion_date_str'] != 'nan') & (df['promotion_date_str'] != ''),
                           ['employee_id', 'promotion_date_str']]

        # Find records where promotion date was before hire date (using original parsed dates)
        promo_before_hire = df[(df['promotion_date_dt'] < df['hire_date_dt']) & df['promotion_date_dt'].notna() & df['hire_date_dt'].notna()].copy()

        # --- Format Records for Diagnostics Output ---
        # Combine bad hire/promo date records
        bad_dates_records = pd.concat([
            bad_hire.assign(type='Hire Date', original_value=bad_hire['hire_date_str']),
            bad_promo.assign(type='Promotion Date', original_value=bad_promo['promotion_date_str'])
        ])[['employee_id', 'type', 'original_value']].to_dict(orient='records')

        # Format promo-before-hire records
        promo_before_hire['hire_date_fmt'] = promo_before_hire['hire_date_dt'].apply(format_timestamp)
        promo_before_hire['promotion_date_fmt'] = promo_before_hire['promotion_date_dt'].apply(format_timestamp)
        # Calculate the original negative TTP for display purposes
        promo_before_hire['orig_ttp'] = ((promo_before_hire['promotion_date_dt'] - promo_before_hire['hire_date_dt']).dt.days / 365.25)

        promo_issue_records = promo_before_hire[['employee_id', 'hire_date_fmt', 'promotion_date_fmt', 'orig_ttp']].round({'orig_ttp': 2}).rename(
            columns={'hire_date_fmt': 'hire_date', 'promotion_date_fmt': 'promotion_date', 'orig_ttp': 'time_to_promotion'}
        ).to_dict(orient='records')

        # Convert diagnostic records to JSON strings
        bad_dates_json = json.dumps(bad_dates_records, default=str) # Use default=str for safety
        promo_issue_json = json.dumps(promo_issue_records, default=str)

        print(f"Found {len(bad_dates_records)} potential date formatting issues.")
        print(f"Found {len(promo_issue_records)} promotion timing issues (promo date before hire date).")

        # --- Create Final Clean DataFrame ---
        # Exclude records with invalid hire dates, as tenure is fundamental
        df_clean = df.dropna(subset=['hire_date_dt']).copy()
        print(f"Proceeding with {len(df_clean)} records having valid hire dates.")

        return df_clean, bad_dates_json, promo_issue_json

    except FileNotFoundError as e:
        print(f"Error: File not found - {e}. Please ensure '{snapshot_file}' and '{history_file}' are in the correct directory.")
        # Return empty structures in case of file error
        return None, "[]", "[]"
    except ValueError as e:
         # Catch specific data validation errors (e.g., missing columns)
         print(f"Error processing data: {e}")
         traceback.print_exc()
         return None, "[]", "[]"
    except Exception as e:
        # Catch any other unexpected errors during loading/processing
        print(f"An unexpected error occurred during data loading: {e}")
        traceback.print_exc()
        return None, "[]", "[]"


# =============================================================================
# # 5. Narrative Generation
# =============================================================================
# Description: This section defines functions to generate textual summaries
# and insights based on the processed data. It includes statistical tests
# (like Chi-square for proportions, t-tests for means) to support the
# observations. The narratives cover workforce composition, tenure, span of
# control, promotions, and organizational design aspects.

# --- Narrative Generation Functions ---

def generate_narrative_summaries(df):
    """
    Generates high-level narrative summaries and performs initial statistical tests.

    Focuses on overall composition, tenure differences, manager ratios, span of
    control, and promotion rates between 'Product Development' and 'Content and Studio'.
    Calculates key metrics and performs Chi-square tests for categorical differences
    and t-tests for numerical differences (tenure, span). Implements OLS regression
    to analyze promotion velocity differences controlling for job level. Synthesizes
    key statistically significant findings. Stores intermediate calculations for
    potential use by more detailed narrative functions. Includes basic checks for
    statistical significance (p < 0.05) and practical significance (e.g., tenure
    difference > 0.5 years). Adds HTML line breaks (<br>) for readability in the
    final dashboard output.

    Args:
        df (pd.DataFrame): The processed DataFrame from `load_data`.

    Returns:
        dict: A dictionary containing narrative strings (e.g., 'composition_summary')
              and intermediate calculation results (e.g., 'avg_tenure').
              Returns {'error': message} if input data is invalid or empty.
    """
    narratives = {}
    intermediate_results = {} # Store calculations for potential reuse

    if df is None or df.empty:
        return {'error': 'Input data is empty or missing.'}

    # --- Data Preparation for Analysis ---
    # Focus analysis on specific job functions, falling back to all data if none found
    comp_df = df[df['job_function'].isin(['Product Development', 'Content and Studio'])].copy()
    if comp_df.empty:
        print("Warning: No data found for 'Product Development' or 'Content and Studio'. Analyzing all data.")
        comp_df = df.copy()

    if comp_df.empty:
         return {'error': 'Filtered or original data is empty.'}

    # --- 1. Composition Summary ---
    try:
        total_employees = len(comp_df)
        intermediate_results['total_employees'] = total_employees

        # Group data by function and role to get counts
        comp_by_func_role = comp_df.groupby(['job_function', 'role'], observed=False).size().unstack(fill_value=0)
        intermediate_results['comp_by_func_role'] = comp_by_func_role

        # Calculate overall percentages for ICs and Managers
        ic_pct = (comp_df['role'] == 'Individual Contributor').mean() * 100 if 'role' in comp_df.columns else np.nan
        mgr_pct = (comp_df['role'] == 'People Manager').mean() * 100 if 'role' in comp_df.columns else np.nan
        intermediate_results['ic_pct'] = ic_pct
        intermediate_results['mgr_pct'] = mgr_pct

        # Calculate manager percentages per function, handling potential missing data
        prod_mgr_pct = np.nan
        cont_mgr_pct = np.nan
        if 'Product Development' in comp_by_func_role.index and 'People Manager' in comp_by_func_role.columns:
            prod_total = comp_by_func_role.loc['Product Development'].sum()
            if prod_total > 0:
                prod_mgr_pct = (comp_by_func_role.loc['Product Development', 'People Manager'] / prod_total) * 100
        if 'Content and Studio' in comp_by_func_role.index and 'People Manager' in comp_by_func_role.columns:
            cont_total = comp_by_func_role.loc['Content and Studio'].sum()
            if cont_total > 0:
                cont_mgr_pct = (comp_by_func_role.loc['Content and Studio', 'People Manager'] / cont_total) * 100
        intermediate_results['prod_mgr_pct'] = prod_mgr_pct
        intermediate_results['cont_mgr_pct'] = cont_mgr_pct

        # Construct the narrative string
        narratives['composition_summary'] = (
            f"Across {total_employees:,} employees analyzed (focusing on Product Development and Content & Studio), "
            f"Individual Contributors constitute {safe_round(ic_pct, 0)}% and People Managers {safe_round(mgr_pct, 0)}%."
            f"<br><br>" # Double line break for spacing in HTML
            f"Manager representation differs: {safe_round(prod_mgr_pct, 0)}% in Product Development versus {safe_round(cont_mgr_pct, 0)}% in Content & Studio. "
            f"This variation might reflect differing organizational structures or team sizes within these functions."
        )

        # --- Statistical Test: Role Distribution (Chi-square) ---
        roles_stat_sig_text = "" # Initialize empty string
        try:
            # Check if data exists for both functions and both roles
            if ('Product Development' in comp_by_func_role.index and
                'Content and Studio' in comp_by_func_role.index and
                'Individual Contributor' in comp_by_func_role.columns and
                'People Manager' in comp_by_func_role.columns):

                # Create the contingency table for the Chi-square test
                contingency_table = comp_by_func_role.loc[['Product Development', 'Content and Studio'], ['Individual Contributor', 'People Manager']]

                # Check if all cell counts are >= 5 (requirement for Chi-square validity)
                if contingency_table.values.min() >= 5:
                    chi2, p_val, _, _ = chi2_contingency(contingency_table)
                    intermediate_results['roles_p_value'] = p_val # Store p-value
                    if p_val < 0.05:
                        roles_stat_sig_text = f"<br><br>A chi-square test (p = {p_val:.3f}) indicates a statistically significant difference in the IC vs. Manager distribution between these two functions."
                    else:
                        roles_stat_sig_text = f"<br><br>A chi-square test (p = {p_val:.3f}) suggests the observed difference in IC vs. Manager distribution may not be statistically significant."
                else:
                     roles_stat_sig_text = "<br><br>Note: Data is insufficient (some group counts < 5) for a reliable chi-square test on role distribution."
                     intermediate_results['roles_p_value'] = np.nan
            else:
                roles_stat_sig_text = "<br><br>Note: Required functions or roles are missing, statistical test for role distribution could not be performed."
                intermediate_results['roles_p_value'] = np.nan
        except Exception as stat_e:
            print(f"Chi-square test for role mix failed: {stat_e}")
            roles_stat_sig_text = "<br><br>Note: An error occurred during the statistical test for role distribution."
            intermediate_results['roles_p_value'] = np.nan

        # Append the statistical significance note to the main summary
        narratives['composition_summary'] += roles_stat_sig_text
        intermediate_results['roles_stat_sig_text'] = roles_stat_sig_text # Store full text

    except Exception as e:
        print(f"Error generating composition narrative: {e}")
        narratives['composition_summary'] = "Error generating composition narrative."
        intermediate_results['roles_stat_sig_text'] = "" # Ensure key exists even on error
        intermediate_results['roles_p_value'] = np.nan

    # --- 2. Tenure Summary ---
    try:
        # Initialize metrics to NaN
        avg_tenure, prod_avg_tenure, cont_avg_tenure = np.nan, np.nan, np.nan
        prod_pct_new, cont_pct_new, diff_tenure = np.nan, np.nan, np.nan

        if 'tenure_years' in comp_df.columns and comp_df['tenure_years'].notna().any():
            # Filter out NaNs and ensure tenure is numeric for calculations
            tenure_df = comp_df[comp_df['tenure_years'].notna()].copy()
            tenure_df['tenure_years'] = pd.to_numeric(tenure_df['tenure_years'], errors='coerce')
            tenure_df.dropna(subset=['tenure_years'], inplace=True)

            if not tenure_df.empty:
                avg_tenure = tenure_df['tenure_years'].mean()
                intermediate_results['avg_tenure'] = avg_tenure

                # Get tenure values for each function
                prod_vals = tenure_df[tenure_df['job_function'] == 'Product Development']['tenure_years']
                cont_vals = tenure_df[tenure_df['job_function'] == 'Content and Studio']['tenure_years']

                prod_avg_tenure = prod_vals.mean() # Returns NaN if prod_vals is empty
                cont_avg_tenure = cont_vals.mean() # Returns NaN if cont_vals is empty
                intermediate_results['prod_avg_tenure'] = prod_avg_tenure
                intermediate_results['cont_avg_tenure'] = cont_avg_tenure

                # Calculate percentage of employees with less than 2 years tenure
                prod_pct_new = (prod_vals < 2).mean() * 100 if not prod_vals.empty else np.nan
                cont_pct_new = (cont_vals < 2).mean() * 100 if not cont_vals.empty else np.nan
                intermediate_results['prod_pct_new'] = prod_pct_new
                intermediate_results['cont_pct_new'] = cont_pct_new

                # Compare average tenures between functions
                if not np.isnan(prod_avg_tenure) and not np.isnan(cont_avg_tenure):
                    diff_tenure = abs(prod_avg_tenure - cont_avg_tenure)
                    intermediate_results['diff_tenure'] = diff_tenure

                    # Narrative based on the magnitude of the difference
                    if diff_tenure < 0.5: # Threshold for similarity
                        comparison = "is broadly similar across functions"
                        narratives['tenure_summary'] = (
                            f"The average employee tenure is {safe_round(avg_tenure, 1)} years and {comparison} "
                            f"(Product Development: {safe_round(prod_avg_tenure, 1)}, Content & Studio: {safe_round(cont_avg_tenure, 1)})."
                            f"<br><br>"
                            f"Employees with less than 2 years of tenure comprise {safe_round(prod_pct_new, 0)}% in Product Development and "
                            f"{safe_round(cont_pct_new, 0)}% in Content & Studio."
                        )
                    else: # Difference is more notable
                        comparison = "differs across functions"
                        narratives['tenure_summary'] = (
                            f"The average employee tenure is {safe_round(avg_tenure, 1)} years, and {comparison}: "
                            f"{safe_round(prod_avg_tenure, 1)} years in Product Development versus {safe_round(cont_avg_tenure, 1)} years in Content & Studio."
                            f"<br><br>"
                            f"The proportion of newer employees (< 2 years tenure) is {safe_round(prod_pct_new, 0)}% in Product Development and "
                            f"{safe_round(cont_pct_new, 0)}% in Content & Studio."
                        )
                else: # Data missing for one or both functions
                     intermediate_results['diff_tenure'] = np.nan
                     narratives['tenure_summary'] = f"The overall average employee tenure is {safe_round(avg_tenure, 1)} years. Data is insufficient for a direct comparison between Product Development and Content & Studio."

                # --- Statistical Test: Tenure Difference (t-test) ---
                tenure_stat_sig_text = ""
                try:
                    # Check for sufficient sample size in both groups (e.g., > 10)
                    if len(prod_vals.dropna()) > 10 and len(cont_vals.dropna()) > 10:
                        # Perform Welch's t-test (robust to unequal variances)
                        t_stat, p_val = ttest_ind(prod_vals.dropna(), cont_vals.dropna(), equal_var=False, nan_policy='omit')
                        intermediate_results['tenure_p_value'] = p_val # Store p-value
                        if p_val < 0.05:
                            tenure_stat_sig_text = f"<br><br>A t-test suggests this difference in average tenure is statistically significant (p = {p_val:.3f}), indicating it is unlikely due to random chance."
                        else:
                            tenure_stat_sig_text = f"<br><br>A t-test indicates the difference in average tenure is not statistically significant (p = {p_val:.3f})."
                    else:
                        tenure_stat_sig_text = "<br><br>Note: Sample sizes (<=10 in one or both groups) are too small to reliably test the statistical significance of the tenure difference."
                        intermediate_results['tenure_p_value'] = np.nan
                except Exception as stat_e:
                     print(f"Error performing tenure significance test: {stat_e}")
                     tenure_stat_sig_text = "<br><br>Note: An error occurred during the statistical test for tenure difference."
                     intermediate_results['tenure_p_value'] = np.nan

                # Append significance test result to tenure summary
                if 'tenure_summary' in narratives: # Ensure base narrative exists
                     narratives['tenure_summary'] += tenure_stat_sig_text
                intermediate_results['tenure_stat_sig_text'] = tenure_stat_sig_text

            else: # If tenure_df is empty after cleaning
                 narratives['tenure_summary'] = "No valid numeric tenure data available for analysis."
                 intermediate_results['tenure_stat_sig_text'] = ""
                 intermediate_results['tenure_p_value'] = np.nan
        else: # If 'tenure_years' column is missing or all NaNs
            narratives['tenure_summary'] = "Tenure data ('tenure_years' column) not available or insufficient for analysis."
            intermediate_results['tenure_stat_sig_text'] = ""
            intermediate_results['tenure_p_value'] = np.nan

        # Ensure intermediate keys exist even if calculations failed
        for key in ['avg_tenure', 'prod_avg_tenure', 'cont_avg_tenure', 'diff_tenure']:
            if key not in intermediate_results: intermediate_results[key] = np.nan
        if 'tenure_stat_sig_text' not in intermediate_results: intermediate_results['tenure_stat_sig_text'] = ""
        if 'tenure_p_value' not in intermediate_results: intermediate_results['tenure_p_value'] = np.nan

    except Exception as e:
        print(f"Error generating tenure narrative: {e}")
        narratives['tenure_summary'] = "Error generating tenure narrative."
        intermediate_results['tenure_stat_sig_text'] = ""
        intermediate_results['tenure_p_value'] = np.nan
        # Ensure intermediate keys exist on error
        for key in ['avg_tenure', 'prod_avg_tenure', 'cont_avg_tenure', 'diff_tenure']:
            if key not in intermediate_results: intermediate_results[key] = np.nan

    # --- 3. Org Design Insight (Manager Ratio) ---
    try:
        # Initialize metrics
        prod_mgr_ratio, cont_mgr_ratio = np.nan, np.nan

        if 'job_function' in comp_df.columns and 'is_manager' in comp_df.columns:
            # Calculate manager counts and total employees per function
            prod_mgrs = comp_df[(comp_df['job_function'] == 'Product Development') & comp_df['is_manager']].shape[0]
            cont_mgrs = comp_df[(comp_df['job_function'] == 'Content and Studio') & comp_df['is_manager']].shape[0]
            prod_total = comp_df[comp_df['job_function'] == 'Product Development'].shape[0]
            cont_total = comp_df[comp_df['job_function'] == 'Content and Studio'].shape[0]

            # Calculate manager ratios (as percentage), handling division by zero
            prod_mgr_ratio = (prod_mgrs / prod_total * 100) if prod_total > 0 else 0
            cont_mgr_ratio = (cont_mgrs / cont_total * 100) if cont_total > 0 else 0
            intermediate_results['prod_mgr_ratio'] = prod_mgr_ratio
            intermediate_results['cont_mgr_ratio'] = cont_mgr_ratio

            # Construct the narrative
            narratives['org_design_insight'] = (
                f"Organizational design appears to differ based on the ratio of managers within each function."
                f"<br><br>Product Development has a manager ratio of {safe_round(prod_mgr_ratio, 1)}%, while Content & Studio has a ratio of {safe_round(cont_mgr_ratio, 1)}%."
                f"<br><br>"
                f"These differences might reflect distinct operational needs or team structures. For instance, technical teams might prioritize a higher ratio of individual contributors for deep technical work, whereas studio teams might require more managers for coordination across diverse creative roles."
            )
        else:
             narratives['org_design_insight'] = "Org design insight requires 'job_function' and 'is_manager' columns."
             intermediate_results['prod_mgr_ratio'] = np.nan
             intermediate_results['cont_mgr_ratio'] = np.nan

    except Exception as e:
        print(f"Error generating org design insight: {e}")
        narratives['org_design_insight'] = "Error generating org design insight."
        intermediate_results['prod_mgr_ratio'] = np.nan
        intermediate_results['cont_mgr_ratio'] = np.nan

    # --- 4. Span of Control Summary ---
    try:
        # Initialize metrics
        avg_span, prod_avg_span, cont_avg_span, diff_span = np.nan, np.nan, np.nan, np.nan

        if 'is_manager' in comp_df.columns and 'span_of_control' in comp_df.columns and 'job_function' in comp_df.columns:
            # Filter for managers with valid, positive span of control data
            managers_df = comp_df[comp_df['is_manager'] & comp_df['span_of_control'].notna() & (comp_df['span_of_control'] > 0)].copy()
            managers_df['span_of_control'] = pd.to_numeric(managers_df['span_of_control'], errors='coerce')
            managers_df.dropna(subset=['span_of_control'], inplace=True)

            if not managers_df.empty:
                avg_span = managers_df['span_of_control'].mean()
                intermediate_results['avg_span'] = avg_span

                # Get span values for each function
                prod_span_vals = managers_df[managers_df['job_function'] == 'Product Development']['span_of_control']
                cont_span_vals = managers_df[managers_df['job_function'] == 'Content and Studio']['span_of_control']

                prod_avg_span = prod_span_vals.mean() # NaN if empty
                cont_avg_span = cont_span_vals.mean() # NaN if empty
                intermediate_results['prod_avg_span'] = prod_avg_span
                intermediate_results['cont_avg_span'] = cont_avg_span

                # Calculate difference if both averages are valid
                if not np.isnan(prod_avg_span) and not np.isnan(cont_avg_span):
                    diff_span = abs(prod_avg_span - cont_avg_span)
                    intermediate_results['diff_span'] = diff_span
                else:
                    intermediate_results['diff_span'] = np.nan

                # --- Statistical Test: Span Difference (t-test) ---
                span_stat_sig_text = ""
                try:
                    if len(prod_span_vals.dropna()) > 10 and len(cont_span_vals.dropna()) > 10:
                        t_stat, p_val = ttest_ind(prod_span_vals.dropna(), cont_span_vals.dropna(), equal_var=False, nan_policy='omit')
                        intermediate_results['span_p_value'] = p_val # Store p-value
                        if p_val < 0.05:
                            span_stat_sig_text = f"<br><br>A t-test suggests this difference in average span is statistically significant (p = {p_val:.3f})."
                        else:
                            span_stat_sig_text = f"<br><br>A t-test indicates the difference in average span may not be statistically significant (p = {p_val:.3f})."
                    else:
                        span_stat_sig_text = "<br><br>Note: Sample sizes (<=10) are too small for reliable span of control significance testing."
                        intermediate_results['span_p_value'] = np.nan
                except Exception as stat_e:
                    print(f"Span t-test failed: {stat_e}")
                    span_stat_sig_text = "<br><br>Note: Statistical test for span difference unavailable due to error."
                    intermediate_results['span_p_value'] = np.nan

                # Construct the narrative
                narratives['span_summary'] = (
                    f"Managers in Product Development have an average span of control of {safe_round(prod_avg_span, 1)} direct reports, "
                    f"compared to {safe_round(cont_avg_span, 1)} in Content & Studio."
                    f"<br><br>"
                    f"The overall average span for managers with direct reports is {safe_round(avg_span, 1)}."
                    f"{span_stat_sig_text}" # Append significance text
                )
                intermediate_results['span_stat_sig_text'] = span_stat_sig_text
            else:
                narratives['span_summary'] = "No managers with valid, positive span of control data were found for analysis."
                intermediate_results['avg_span'] = np.nan
                intermediate_results['prod_avg_span'] = np.nan
                intermediate_results['cont_avg_span'] = np.nan
                intermediate_results['diff_span'] = np.nan
                intermediate_results['span_stat_sig_text'] = ""
                intermediate_results['span_p_value'] = np.nan
        else:
             narratives['span_summary'] = "Span summary requires 'is_manager', 'span_of_control', and 'job_function' columns."
             intermediate_results['avg_span'] = np.nan
             intermediate_results['prod_avg_span'] = np.nan
             intermediate_results['cont_avg_span'] = np.nan
             intermediate_results['diff_span'] = np.nan
             intermediate_results['span_stat_sig_text'] = ""
             intermediate_results['span_p_value'] = np.nan

    except Exception as e:
        print(f"Error generating span summary: {e}")
        narratives['span_summary'] = "Error generating span summary."
        intermediate_results['avg_span'] = np.nan
        intermediate_results['prod_avg_span'] = np.nan
        intermediate_results['cont_avg_span'] = np.nan
        intermediate_results['diff_span'] = np.nan
        intermediate_results['span_stat_sig_text'] = ""
        intermediate_results['span_p_value'] = np.nan

    # --- 5. Promotion Insight (Rate by Function) ---
    try:
        # Initialize metrics
        promo_rate_prod, promo_rate_cont = np.nan, np.nan
        promo_counts = pd.Series(dtype=int)
        total_by_func = pd.Series(dtype=int)

        if 'is_promoted' in comp_df.columns and 'job_function' in comp_df.columns:
            # Filter for employees with valid promotion records
            promo_df = comp_df[comp_df['is_promoted']].copy()
            if not promo_df.empty:
                # Calculate total employees per function
                total_by_func = comp_df.groupby('job_function', observed=False).size()
                # Calculate promoted employees per function
                promo_counts = promo_df.groupby('job_function', observed=False).size()
                intermediate_results['promo_counts'] = promo_counts
                intermediate_results['total_by_func'] = total_by_func

                # Get counts, defaulting to 0 if function doesn't exist
                prod_total = total_by_func.get('Product Development', 0)
                cont_total = total_by_func.get('Content and Studio', 0)
                prod_promoted = promo_counts.get('Product Development', 0)
                cont_promoted = promo_counts.get('Content and Studio', 0)

                # Calculate promotion rates (as percentage), handling division by zero
                promo_rate_prod = (prod_promoted / prod_total * 100) if prod_total > 0 else 0
                promo_rate_cont = (cont_promoted / cont_total * 100) if cont_total > 0 else 0
                intermediate_results['promo_rate_prod'] = promo_rate_prod
                intermediate_results['promo_rate_cont'] = promo_rate_cont

                # --- Statistical Test: Promotion Rates (Chi-square) ---
                promo_rate_stat_sig_text = ""
                try:
                    # Ensure totals are non-zero before creating contingency table
                    if prod_total > 0 and cont_total > 0:
                        # Calculate non-promoted counts
                        prod_not_promoted = prod_total - prod_promoted
                        cont_not_promoted = cont_total - cont_promoted
                        contingency_table = [[prod_promoted, prod_not_promoted],
                                             [cont_promoted, cont_not_promoted]]

                        # Check for sufficient counts (>= 5 in all cells)
                        if all(count >= 5 for row in contingency_table for count in row):
                            chi2, p_val, _, _ = chi2_contingency(contingency_table)
                            intermediate_results['promo_rate_p_value'] = p_val # Store p-value
                            if p_val < 0.05:
                                promo_rate_stat_sig_text = f"<br><br>A chi-square test (p = {p_val:.3f}) suggests this difference in promotion rates is statistically significant."
                            else:
                                promo_rate_stat_sig_text = f"<br><br>A chi-square test (p = {p_val:.3f}) indicates the difference in promotion rates may not be statistically significant."
                        else:
                            promo_rate_stat_sig_text = "<br><br>Note: Data is insufficient (some group counts < 5) for a reliable chi-square test on promotion rates."
                            intermediate_results['promo_rate_p_value'] = np.nan
                    else:
                        promo_rate_stat_sig_text = "<br><br>Note: Cannot perform chi-square test on promotion rates as one or both functions have zero total employees."
                        intermediate_results['promo_rate_p_value'] = np.nan

                except Exception as stat_e:
                    print(f"Promotion rate Chi-square test failed: {stat_e}")
                    promo_rate_stat_sig_text = "<br><br>Note: Statistical test for promotion rates unavailable due to error."
                    intermediate_results['promo_rate_p_value'] = np.nan

                # Construct the narrative
                narratives['promotion_insight'] = (
                    f"Overall promotion rates observed in the data are: "
                    f"Product Development at {safe_round(promo_rate_prod, 1)}% and Content & Studio at {safe_round(promo_rate_cont, 1)}%."
                    f"<br><br>"
                    f"Differences in promotion rates could reflect variations in growth trajectories, role structures, or performance cycles between the functions."
                    f"{promo_rate_stat_sig_text}" # Append significance text
                )
                intermediate_results['promo_rate_stat_sig_text'] = promo_rate_stat_sig_text
            else:
                narratives['promotion_insight'] = "No employees with valid promotion records were found in the data to analyze rates."
                intermediate_results['promo_rate_prod'] = np.nan
                intermediate_results['promo_rate_cont'] = np.nan
                intermediate_results['promo_rate_stat_sig_text'] = ""
                intermediate_results['promo_rate_p_value'] = np.nan
        else:
            narratives['promotion_insight'] = "Promotion insight requires 'is_promoted' and 'job_function' columns."
            intermediate_results['promo_rate_prod'] = np.nan
            intermediate_results['promo_rate_cont'] = np.nan
            intermediate_results['promo_rate_stat_sig_text'] = ""
            intermediate_results['promo_rate_p_value'] = np.nan

    except Exception as e:
        print(f"Error generating promotion insight: {e}")
        narratives['promotion_insight'] = "Error generating promotion insight."
        intermediate_results['promo_rate_prod'] = np.nan
        intermediate_results['promo_rate_cont'] = np.nan
        intermediate_results['promo_rate_stat_sig_text'] = ""
        intermediate_results['promo_rate_p_value'] = np.nan

    # --- 6. Promotions Trend Summary (by Year) ---
    try:
        if 'is_promoted' in comp_df.columns and 'promotion_year' in comp_df.columns:
            # Filter for promoted employees with valid, numeric promotion year
            promo_trend_df = comp_df[comp_df['is_promoted'] & comp_df['promotion_year'].notna()].copy()
            promo_trend_df['promotion_year'] = pd.to_numeric(promo_trend_df['promotion_year'], errors='coerce')
            promo_trend_df.dropna(subset=['promotion_year'], inplace=True)

            if not promo_trend_df.empty:
                # Group by year and count promotions
                trend = promo_trend_df.groupby('promotion_year').size()
                if not trend.empty:
                    peak_year = trend.idxmax() # Year with the highest number of promotions
                    narratives['promotions_trend_summary'] = (
                        f"Analysis of promotion data shows activity peaked in {int(peak_year)} with {trend[peak_year]:,} recorded promotions."
                        f"<br><br>Year-over-year trends in promotion volume can be influenced by factors such as large hiring cohorts reaching promotion eligibility, changes in promotion policies, or shifts in business growth and investment."
                    )
                else:
                     narratives['promotions_trend_summary'] = "No promotion data found after grouping by year."
            else:
                narratives['promotions_trend_summary'] = "No valid promotion data (is_promoted=True and numeric non-null promotion_year) available for trend analysis."
        else:
            narratives['promotions_trend_summary'] = "Promotion trend summary requires 'is_promoted' and 'promotion_year' columns."

    except Exception as e:
        print(f"Error generating promotions trend summary: {e}")
        narratives['promotions_trend_summary'] = "Error generating promotions trend summary."

    # --- 7. Regression-based Insight: Time to Promotion ---
    # Reimplemented OLS regression logic
    try:
        # Check for necessary columns
        required_cols = ['time_to_promotion', 'job_function', 'job_level_title']
        if all(col in comp_df.columns for col in required_cols):
            # Prepare data: drop NaNs in relevant columns first
            reg_df = comp_df[required_cols].dropna().copy()

            # Convert time_to_promotion to numeric *after* initial dropna
            reg_df['time_to_promotion'] = pd.to_numeric(reg_df['time_to_promotion'], errors='coerce')
            reg_df.dropna(subset=['time_to_promotion'], inplace=True) # Drop if conversion failed

            # Check if sufficient data remains (e.g., > 50 for reliability)
            if len(reg_df) > 50:
                try:
                    # Create dummy variables for categorical features
                    # drop_first=True avoids multicollinearity
                    reg_df_dummies = pd.get_dummies(reg_df, columns=['job_function', 'job_level_title'], drop_first=True, dummy_na=False, dtype=float)

                    # Define independent variables (X) and dependent variable (y)
                    y = reg_df_dummies['time_to_promotion']
                    X = reg_df_dummies.drop(columns=['time_to_promotion'])
                    X = sm.add_constant(X, has_constant='add') # Add intercept

                    # Ensure X and y are purely numeric and aligned (handle potential NaNs introduced by dummies/constant)
                    X = X.astype(float) # Ensure all X columns are float
                    y = y.astype(float)
                    combined = pd.concat([y, X], axis=1).dropna() # Drop rows with any NaNs

                    if len(combined) > 10: # Check if enough data remains after cleaning
                        y_clean = combined['time_to_promotion']
                        X_clean = combined.drop(columns=['time_to_promotion'])

                        # Fit the Ordinary Least Squares (OLS) model
                        model = sm.OLS(y_clean, X_clean).fit()
                        intermediate_results['ttp_regression_model'] = model # Store model results

                        # Check if the specific predictor exists (depends on base category)
                        # Assumes 'Content and Studio' is the base category for job_function
                        predictor_name = 'job_function_Product Development'
                        if predictor_name in model.params:
                            beta = model.params[predictor_name] # Coefficient
                            pval = model.pvalues[predictor_name] # P-value
                            intermediate_results['ttp_regression_beta'] = beta
                            intermediate_results['ttp_regression_pval'] = pval

                            if pval < 0.05: # Check for statistical significance
                                direction = 'faster' if beta < 0 else 'slower'
                                narratives['promotion_regression_insight'] = (
                                    f"Controlling for job level, being in Product Development is associated with statistically significantly {direction} promotions (average difference of {abs(beta):.2f} years, p = {pval:.3f})."
                                    f"<br><br>This suggests that function-specific factors, beyond just the assigned job level, may influence the typical time it takes for an employee to be promoted in these areas."
                                )
                            else:
                                narratives['promotion_regression_insight'] = (
                                    f"After controlling for job level, the analysis did not find a statistically significant difference (p = {pval:.3f}) in the average time to promotion between Product Development and Content & Studio."
                                )
                        else:
                            narratives['promotion_regression_insight'] = f"Regression analysis completed, but the specific predictor '{predictor_name}' was not included in the final model, possibly due to data characteristics (e.g., collinearity or lack of variation)."
                            intermediate_results['ttp_regression_beta'] = np.nan
                            intermediate_results['ttp_regression_pval'] = np.nan
                    else:
                         narratives['promotion_regression_insight'] = "Insufficient data remaining after cleaning numeric types required for regression analysis."
                         intermediate_results['ttp_regression_beta'] = np.nan
                         intermediate_results['ttp_regression_pval'] = np.nan
                except Exception as model_e:
                     print(f"Error during regression model fitting or analysis: {model_e}")
                     traceback.print_exc()
                     narratives['promotion_regression_insight'] = f"An error occurred during the regression analysis: {model_e}"
                     intermediate_results['ttp_regression_beta'] = np.nan
                     intermediate_results['ttp_regression_pval'] = np.nan
            else:
                narratives['promotion_regression_insight'] = f"Insufficient data ({len(reg_df)} valid rows) to run regression reliably after handling missing values."
                intermediate_results['ttp_regression_beta'] = np.nan
                intermediate_results['ttp_regression_pval'] = np.nan
        else:
            narratives['promotion_regression_insight'] = f"Regression insight requires columns: {', '.join(required_cols)}."
            intermediate_results['ttp_regression_beta'] = np.nan
            intermediate_results['ttp_regression_pval'] = np.nan

    except Exception as e:
        print(f"Error generating promotion regression insight setup: {e}")
        traceback.print_exc()
        narratives['promotion_regression_insight'] = "Error setting up promotion regression insight."
        intermediate_results['ttp_regression_beta'] = np.nan
        intermediate_results['ttp_regression_pval'] = np.nan

    # --- 8. Executive Summary Findings Synthesis ---
    # Reimplemented dynamic synthesis based on intermediate results
    try:
        exec_points = []
        # Retrieve results from intermediate storage
        diff_tenure_local = intermediate_results.get('diff_tenure', np.nan)
        tenure_pval_local = intermediate_results.get('tenure_p_value', np.nan)
        prod_avg_tenure_local = intermediate_results.get('prod_avg_tenure', np.nan)
        cont_avg_tenure_local = intermediate_results.get('cont_avg_tenure', np.nan)

        diff_span_local = intermediate_results.get('diff_span', np.nan)
        span_pval_local = intermediate_results.get('span_p_value', np.nan)
        prod_avg_span_local = intermediate_results.get('prod_avg_span', np.nan)
        cont_avg_span_local = intermediate_results.get('cont_avg_span', np.nan)

        promo_rate_pval_local = intermediate_results.get('promo_rate_p_value', np.nan)
        promo_rate_prod_local = intermediate_results.get('promo_rate_prod', np.nan)
        promo_rate_cont_local = intermediate_results.get('promo_rate_cont', np.nan)

        ttp_regr_beta_local = intermediate_results.get('ttp_regression_beta', np.nan)
        ttp_regr_pval_local = intermediate_results.get('ttp_regression_pval', np.nan)

        # Check Tenure Significance (p < 0.05 and difference >= 0.5 years)
        if not np.isnan(tenure_pval_local) and tenure_pval_local < 0.05 and not np.isnan(diff_tenure_local) and diff_tenure_local >= 0.5:
             exec_points.append(f"Average tenure differs significantly (p={tenure_pval_local:.3f}) between Product Development ({safe_round(prod_avg_tenure_local, 1)} yrs) and Content & Studio ({safe_round(cont_avg_tenure_local, 1)} yrs).")

        # Check Span Significance (p < 0.05 and difference >= 1.0 report)
        if not np.isnan(span_pval_local) and span_pval_local < 0.05 and not np.isnan(diff_span_local) and diff_span_local >= 1.0:
            exec_points.append(f"Average manager span of control differs significantly (p={span_pval_local:.3f}) between Product Development ({safe_round(prod_avg_span_local, 1)}) and Content & Studio ({safe_round(cont_avg_span_local, 1)}).")

        # Check Promotion Rate Significance (p < 0.05)
        if not np.isnan(promo_rate_pval_local) and promo_rate_pval_local < 0.05:
             exec_points.append(f"Promotion rates differ significantly (p={promo_rate_pval_local:.3f}): Product Development ({safe_round(promo_rate_prod_local, 1)}%) vs Content & Studio ({safe_round(promo_rate_cont_local, 1)}%).")

        # Check Promotion Regression Significance (p < 0.05)
        if not np.isnan(ttp_regr_pval_local) and ttp_regr_pval_local < 0.05:
             direction = 'faster' if ttp_regr_beta_local < 0 else 'slower'
             years_diff = abs(ttp_regr_beta_local)
             exec_points.append(f"After controlling for level, Product Development promotes significantly {direction} (by {years_diff:.2f} yrs, p={ttp_regr_pval_local:.3f}) compared to Content & Studio.")

        # Construct the final summary message
        if exec_points:
            # Use HTML list for better formatting
            findings_list = "".join([f"<li>{point}</li>" for point in exec_points])
            narratives['exec_summary_findings'] = (
                "Key statistically significant differences (p < 0.05) meeting analysis thresholds were observed between functions:"
                f"<ul>{findings_list}</ul>"
            )
        else:
            narratives['exec_summary_findings'] = "No statistically significant differences meeting the defined thresholds were found in the comparisons of tenure, span of control, promotion rates, or promotion timing (controlling for level) between Product Development and Content & Studio."

    except Exception as e:
        print(f"Error synthesizing exec findings: {e}")
        narratives['exec_summary_findings'] = "Error synthesizing executive summary findings."

    # Add intermediate results to the main narratives dictionary
    narratives.update(intermediate_results)

    return narratives

# --- Detailed Narrative Generation Function ---

def generate_detailed_narratives(df, bad_dates_json_str="[]", promo_issue_json_str="[]"):
    """
    Generates detailed narrative insights matching the dashboard template keys.

    Builds upon the output of `generate_narrative_summaries` by adding more
    granular insights, formatting text with HTML breaks (<br>), and structuring
    content for specific dashboard sections (Composition, Tenure, Span, Promotions,
    Hiring, Time to Promotion, Diagnostics, Strategic). Includes data quality
    summaries based on provided JSON strings and generates conditional strategic
    recommendations.

    Args:
        df (pd.DataFrame): The primary processed DataFrame from `load_data`.
        bad_dates_json_str (str, optional): JSON string listing records with
                                             date formatting issues. Defaults to "[]".
        promo_issue_json_str (str, optional): JSON string listing records with
                                              promotion timing issues. Defaults to "[]".

    Returns:
        dict: A dictionary containing narrative strings keyed to match the
              HTML template's expected variables. Returns the initial error
              dictionary if `generate_narrative_summaries` failed.
    """
    # Start with the basic summaries and intermediate results from Part 1 function
    # This dictionary already contains initial narratives and calculation results.
    narratives = generate_narrative_summaries(df)

    # Exit early if the initial summary generation failed
    if 'error' in narratives:
        print("Error detected in initial summary generation. Skipping detailed narratives.")
        # Clean up intermediate keys potentially added before the error
        keys_to_remove = {k for k in narratives if k != 'error'}
        for key in keys_to_remove:
            narratives.pop(key, None)
        return narratives

    # Re-filter data consistently (focus on Product Dev and Content/Studio)
    comp_df = df[df['job_function'].isin(['Product Development', 'Content and Studio'])].copy()
    if comp_df.empty:
        # Fallback to original df if filtering removes all data
        comp_df = df.copy()

    if comp_df.empty:
         print("Warning: No data available for detailed narrative generation after filtering.")
         # Return the narratives dict which might contain summaries based on full data if fallback occurred
         return narratives # Return summaries generated so far

    # --- Generate Detailed Narratives for Template Keys ---

    # == Composition Section ==
    # 'composition_summary': Already generated by generate_narrative_summaries.
    # 'org_design_insight': Already generated by generate_narrative_summaries.

    # --- Roles Insight (Detailed IC/Manager Mix) ---
    try:
        if 'job_function' in comp_df.columns and 'role' in comp_df.columns:
            # Calculate role percentages per function
            roles_by_func = comp_df.groupby(['job_function', 'role'], observed=False).size().unstack(fill_value=0)
            roles_pct = roles_by_func.apply(lambda x: x / x.sum() * 100 if x.sum() > 0 else x, axis=1)

            # Extract percentages, handling missing data
            prod_ic_pct = roles_pct.loc['Product Development', 'Individual Contributor'] if ('Product Development' in roles_pct.index and 'Individual Contributor' in roles_pct.columns) else np.nan
            cont_ic_pct = roles_pct.loc['Content and Studio', 'Individual Contributor'] if ('Content and Studio' in roles_pct.index and 'Individual Contributor' in roles_pct.columns) else np.nan

            # Retrieve the statistical significance text generated earlier
            roles_sig_text = narratives.get('roles_stat_sig_text', ' Statistical test result unavailable.')

            # Construct the narrative
            narratives['roles_insight'] = (
                f"Product Development and Content & Studio exhibit distinct ratios of Individual Contributors (ICs) to People Managers."
                f"<br><br>"
                f"Product Development comprises {safe_round(prod_ic_pct, 0)}% ICs, compared to {safe_round(cont_ic_pct, 0)}% in Content & Studio."
                f"<br>" # Single break before the interpretation
                f"This difference may reflect fundamental variations in how technical work (potentially favoring deeper IC roles) and creative/studio work (potentially requiring more coordination layers) are structured."
                f"{roles_sig_text}" # Append the pre-formatted statistical test result
            )
        else:
             narratives['roles_insight'] = "Roles insight requires 'job_function' and 'role' columns with valid data."
    except Exception as e:
        print(f"Error generating roles_insight: {e}")
        narratives['roles_insight'] = "Error generating roles insight."

    # --- Leadership Insight (Director/VP Distribution) ---
    try:
        if 'job_level_title' in comp_df.columns and 'job_function' in comp_df.columns:
            leadership_levels = ['Director', 'Vice President']
            leaders_df = comp_df[comp_df['job_level_title'].isin(leadership_levels)]

            if not leaders_df.empty:
                # Count leaders in each target function
                prod_leaders = leaders_df[leaders_df['job_function'] == 'Product Development'].shape[0]
                cont_leaders = leaders_df[leaders_df['job_function'] == 'Content and Studio'].shape[0]
                total_leaders_in_scope = prod_leaders + cont_leaders

                # Calculate percentage distribution *among these leaders*
                prod_pct_of_leaders = (prod_leaders / total_leaders_in_scope * 100) if total_leaders_in_scope > 0 else 0
                cont_pct_of_leaders = (cont_leaders / total_leaders_in_scope * 100) if total_leaders_in_scope > 0 else 0

                # --- Statistical Test: Proportion of Leaders (Chi-square) ---
                leader_prop_stat_text = ""
                try:
                    # Get total employees in each function for the test
                    prod_total_func = comp_df[comp_df['job_function'] == 'Product Development'].shape[0]
                    cont_total_func = comp_df[comp_df['job_function'] == 'Content and Studio'].shape[0]

                    if prod_total_func > 0 and cont_total_func > 0:
                        # Calculate non-leaders in each function
                        prod_non_leaders = prod_total_func - prod_leaders
                        cont_non_leaders = cont_total_func - cont_leaders
                        # Contingency table: [Leaders, Non-Leaders] x [Prod, Content]
                        contingency = [[prod_leaders, prod_non_leaders], [cont_leaders, cont_non_leaders]]

                        # Check counts for validity
                        if all(count >= 5 for row in contingency for count in row):
                            chi2, p_val, _, _ = chi2_contingency(contingency)
                            significance = "differs significantly" if p_val < 0.05 else "does not differ significantly"
                            leader_prop_stat_text = f"<br><br>A chi-square test (p = {p_val:.3f}) suggests the proportion of senior leaders ({', '.join(leadership_levels)}) {significance} between these functions."
                        else:
                            leader_prop_stat_text = f"<br><br>Note: Data is insufficient (some group counts < 5) to reliably test the significance of the leadership proportion difference."
                    else:
                        leader_prop_stat_text = "<br><br>Note: Cannot perform statistical test on leadership proportion as one or both functions have zero employees."
                except Exception as stat_e:
                    print(f"Chi-square test for leadership proportion failed: {stat_e}")
                    leader_prop_stat_text = "<br><br>Note: An error occurred during the statistical test for leadership proportion."

                # Construct the narrative
                narratives['leadership_insight'] = (
                    f"Among the senior leaders ({', '.join(leadership_levels)}) specifically within Product Development and Content & Studio, "
                    f"{safe_round(prod_pct_of_leaders, 0)}% are affiliated with Product Development, while {safe_round(cont_pct_of_leaders, 0)}% are with Content & Studio."
                    f"<br><br>"
                    f"This distribution may indicate differences in the relative scale or hierarchical structure of leadership within these two core functions."
                    f"{leader_prop_stat_text}" # Append significance text
                )
            else:
                 narratives['leadership_insight'] = f"No employees were found at the specified leadership levels ({', '.join(leadership_levels)}) within the analyzed functions."
        else:
            narratives['leadership_insight'] = "Leadership insight requires 'job_level_title' and 'job_function' columns with valid data."
    except Exception as e:
        print(f"Error generating leadership_insight: {e}")
        narratives['leadership_insight'] = "Error generating leadership insight."


    # == Tenure Section ==
    # 'tenure_summary': Already generated (Average Tenure / Density).

    # --- Detailed Tenure Insight (Median and Long Tenure) ---
    try:
        if 'tenure_years' in comp_df.columns and comp_df['tenure_years'].notna().any():
            # Use NaN-filtered, numeric tenure data
            tenure_df = comp_df[comp_df['tenure_years'].notna()].copy()
            tenure_df['tenure_years'] = pd.to_numeric(tenure_df['tenure_years'], errors='coerce')
            tenure_df.dropna(subset=['tenure_years'], inplace=True)

            if not tenure_df.empty:
                # Calculate median tenure per function
                prod_median = tenure_df[tenure_df['job_function'] == 'Product Development']['tenure_years'].median()
                cont_median = tenure_df[tenure_df['job_function'] == 'Content and Studio']['tenure_years'].median()

                # Calculate percentage with 4+ years tenure per function
                prod_4yr_pct = (tenure_df[tenure_df['job_function'] == 'Product Development']['tenure_years'] >= 4).mean() * 100
                cont_4yr_pct = (tenure_df[tenure_df['job_function'] == 'Content and Studio']['tenure_years'] >= 4).mean() * 100

                # Compare medians if both are valid numbers
                if not np.isnan(prod_median) and not np.isnan(cont_median):
                    median_diff = abs(prod_median - cont_median)
                    if median_diff < 0.5: # Threshold for similarity
                        narratives['tenure_insight'] = (
                            f"Median tenure shows similarity across functions (Product Development: {safe_round(prod_median, 1)} years, Content & Studio: {safe_round(cont_median, 1)} years)."
                            f"<br><br>"
                            f"The proportion of employees with four or more years of tenure is {safe_round(prod_4yr_pct, 0)}% in Product Development and {safe_round(cont_4yr_pct, 0)}% in Content & Studio."
                        )
                    else: # Notable difference
                        narratives['tenure_insight'] = (
                            f"Median tenure differs between functions: Product Development is at {safe_round(prod_median, 1)} years, while Content & Studio is at {safe_round(cont_median, 1)} years."
                            f"<br><br>"
                            f"Regarding longer-serving employees (4+ years tenure), they constitute {safe_round(prod_4yr_pct, 0)}% of Product Development and {safe_round(cont_4yr_pct, 0)}% of Content & Studio."
                        )
                else: # Handle cases where median calculation failed for one/both
                     narratives['tenure_insight'] = "Median tenure comparison is not possible due to missing or insufficient data in one or both functions."
            else:
                narratives['tenure_insight'] = "No valid numeric tenure data available for detailed insight calculation."
        else:
            narratives['tenure_insight'] = "Detailed tenure insight requires the 'tenure_years' column with valid data."
    except Exception as e:
        print(f"Error generating detailed tenure insight: {e}")
        narratives['tenure_insight'] = "Error generating detailed tenure insight."

    # --- Tenure by Level Insight ---
    try:
        if 'tenure_years' in comp_df.columns and 'job_level_title' in comp_df.columns:
            # Use NaN-filtered, numeric tenure data
            tenure_level_df = comp_df[comp_df['tenure_years'].notna()].copy()
            tenure_level_df['tenure_years'] = pd.to_numeric(tenure_level_df['tenure_years'], errors='coerce')
            tenure_level_df.dropna(subset=['tenure_years'], inplace=True)

            if not tenure_level_df.empty:
                # Group by cleaned job level title and calculate mean tenure
                avg_tenure_by_level = tenure_level_df.groupby('job_level_title')['tenure_years'].mean().round(1)
                # Exclude 'Unknown' level and sort by tenure (descending)
                avg_tenure_by_level = avg_tenure_by_level[avg_tenure_by_level.index != 'Unknown'].sort_values(ascending=False)

                if not avg_tenure_by_level.empty:
                    # Format as an HTML list for the narrative
                    level_tenure_items = [f"<li>{lvl}: {val} years</li>" for lvl, val in avg_tenure_by_level.items()]
                    level_tenure_formatted_list = f"<ul>{''.join(level_tenure_items)}</ul>"
                    narratives['tenure_level_insight'] = (
                        f"Average tenure generally increases with job level, reflecting experience accumulation. Representative average tenures include:"
                        f"{level_tenure_formatted_list}"
                    )
                else:
                    narratives['tenure_level_insight'] = "Could not calculate average tenure by level after filtering for known levels."
            else:
                narratives['tenure_level_insight'] = "No valid numeric tenure data found to analyze tenure by job level."
        else:
            narratives['tenure_level_insight'] = "Tenure by level insight requires 'tenure_years' and 'job_level_title' columns with valid data."
    except Exception as e:
        print(f"Error generating tenure_level_insight: {e}")
        narratives['tenure_level_insight'] = "Error generating tenure by level insight."


    # == Span Section ==
    # 'span_summary': Already generated (Average Span Comparison).

    # --- Span by Level Insight ---
    try:
        if 'span_of_control' in comp_df.columns and 'job_level_title' in comp_df.columns:
            # Filter for managers with valid, positive span
            level_span_df = comp_df[comp_df['is_manager'] & comp_df['span_of_control'].notna() & (comp_df['span_of_control'] > 0)].copy()
            level_span_df['span_of_control'] = pd.to_numeric(level_span_df['span_of_control'], errors='coerce')
            level_span_df.dropna(subset=['span_of_control'], inplace=True)

            if not level_span_df.empty:
                # Group by level and calculate average span
                avg_span_by_level = level_span_df.groupby('job_level_title')['span_of_control'].mean().round(1)
                # Filter for typical manager levels and sort
                manager_levels_ordered = ['Manager', 'Director', 'Vice President']
                avg_span_by_level = avg_span_by_level.reindex(manager_levels_ordered).dropna()

                if not avg_span_by_level.empty:
                    # Format as HTML list
                    level_span_items = [f"<li>{level}: {val} reports</li>" for level, val in avg_span_by_level.items()]
                    level_span_formatted_list = f"<ul>{''.join(level_span_items)}</ul>"
                    narratives['span_level_insight'] = (
                        f"Average span of control tends to increase with managerial level. Sample averages for managers with direct reports:"
                        f"{level_span_formatted_list}"
                    )
                else:
                    narratives['span_level_insight'] = "Could not calculate average span by manager level after filtering."
            else:
                narratives['span_level_insight'] = "No managers with valid, positive span of control data were found."
        else:
            narratives['span_level_insight'] = "Span by level insight requires 'span_of_control', 'job_level_title', and 'is_manager' columns with valid data."
    except Exception as e:
        print(f"Error generating span_level_insight: {e}")
        narratives['span_level_insight'] = "Error generating span by level insight."

    # --- Combined Span Control Insight ---
    # This combines the average span comparison and the span-by-level insight.
    try:
        span_summary_text = narratives.get('span_summary', "")
        span_level_text = narratives.get('span_level_insight', "")
        combined_span_text = ""

        # Add summary text if valid
        if span_summary_text and not span_summary_text.startswith("Error") and not span_summary_text.startswith("No managers"):
            combined_span_text += span_summary_text

        # Add level text if valid, adding spacing
        if span_level_text and not span_level_text.startswith("Error") and not span_level_text.startswith("Could not") and not span_level_text.startswith("No managers"):
             if combined_span_text: # Add breaks if combining with previous text
                 combined_span_text += f"<br><br>Furthermore, {span_level_text}"
             else:
                 combined_span_text = span_level_text # Use directly if it's the only text

        if combined_span_text:
             narratives['span_control_insight'] = combined_span_text
        else:
             # Provide a fallback if both underlying insights failed
             narratives['span_control_insight'] = "Detailed span of control insights could not be generated due to missing or insufficient data."

    except Exception as e:
        print(f"Error generating span_control_insight: {e}")
        narratives['span_control_insight'] = "Error combining span insights."


    # == Promotions Section ==
    # 'promotion_insight': Already generated (Rate Comparison).
    # 'promotions_trend_summary': Already generated (Peak Year).

    # --- Promotion Seasonality Insight ---
    try:
        if 'promotion_date_dt' in comp_df.columns and 'job_function' in comp_df.columns:
            # Ensure the date column is actually datetime type
            if pd.api.types.is_datetime64_any_dtype(comp_df['promotion_date_dt']):
                # Filter for records with valid promotion dates
                promo_quarters_df = comp_df[comp_df['promotion_date_dt'].notna()].copy()
                if not promo_quarters_df.empty:
                    # Extract quarter and group by function/quarter
                    promo_quarters_df['promo_q'] = 'Q' + promo_quarters_df['promotion_date_dt'].dt.quarter.astype(str)
                    promo_q_counts = promo_quarters_df.groupby(['job_function', 'promo_q'], observed=False).size().unstack(fill_value=0)

                    # Determine peak quarter for each function
                    prod_peak_q_str = ""
                    cont_peak_q_str = ""
                    if 'Product Development' in promo_q_counts.index and promo_q_counts.loc['Product Development'].sum() > 0:
                        prod_peak_q = promo_q_counts.loc['Product Development'].idxmax()
                        prod_peak_q_str = f"{prod_peak_q} for Product Development"
                    if 'Content and Studio' in promo_q_counts.index and promo_q_counts.loc['Content and Studio'].sum() > 0:
                        cont_peak_q = promo_q_counts.loc['Content and Studio'].idxmax()
                        cont_peak_q_str = f"{cont_peak_q} for Content & Studio"

                    # Construct narrative based on available peak data
                    if prod_peak_q_str and cont_peak_q_str:
                         narratives['promotion_seasonality_insight'] = f"Promotion activity exhibits seasonality, tending to peak in {prod_peak_q_str} and {cont_peak_q_str}." \
                                                                        f"<br><br>This pattern may align with annual performance review cycles, planning periods, or other business cadences."
                    elif prod_peak_q_str:
                         narratives['promotion_seasonality_insight'] = f"Promotion activity tends to peak in {prod_peak_q_str}. Data for Content & Studio peak was unavailable or insufficient."
                    elif cont_peak_q_str:
                         narratives['promotion_seasonality_insight'] = f"Promotion activity tends to peak in {cont_peak_q_str}. Data for Product Development peak was unavailable or insufficient."
                    else:
                         narratives['promotion_seasonality_insight'] = "Insufficient promotion data available to determine peak quarters for either function."
                else:
                    narratives['promotion_seasonality_insight'] = "No valid promotion dates found for seasonality analysis."
            else:
                 narratives['promotion_seasonality_insight'] = "Promotion seasonality analysis requires the 'promotion_date_dt' column to be of datetime type."
        else:
            narratives['promotion_seasonality_insight'] = "Promotion seasonality insight requires 'promotion_date_dt' and 'job_function' columns."
    except Exception as e:
        print(f"Error generating promotion_seasonality_insight: {e}")
        narratives['promotion_seasonality_insight'] = "Error generating promotion seasonality insight."


    # == Hiring Section ==

    # --- Hiring Year Insight (Year-over-Year Change) ---
    try:
        if 'hire_year' in comp_df.columns and 'job_function' in comp_df.columns:
            # Ensure hire_year is numeric and drop NaNs
            hire_years_df = comp_df[comp_df['hire_year'].notna()].copy()
            hire_years_df['hire_year'] = pd.to_numeric(hire_years_df['hire_year'], errors='coerce')
            hire_years_df.dropna(subset=['hire_year'], inplace=True)
            hire_years_df['hire_year'] = hire_years_df['hire_year'].astype(int) # Convert to int after dropna

            if not hire_years_df.empty:
                # Group hires by year and function
                hires_by_year_func = hire_years_df.groupby(['hire_year', 'job_function'], observed=False).size().unstack(fill_value=0)

                if len(hires_by_year_func) >= 2:
                    # Compare the two most recent years with data
                    hires_by_year_func = hires_by_year_func.sort_index()
                    year2 = hires_by_year_func.index[-1]
                    year1 = hires_by_year_func.index[-2]

                    # Get counts for each function in the two years
                    prod_y1 = hires_by_year_func.loc[year1, 'Product Development'] if 'Product Development' in hires_by_year_func.columns else 0
                    prod_y2 = hires_by_year_func.loc[year2, 'Product Development'] if 'Product Development' in hires_by_year_func.columns else 0
                    cont_y1 = hires_by_year_func.loc[year1, 'Content and Studio'] if 'Content and Studio' in hires_by_year_func.columns else 0
                    cont_y2 = hires_by_year_func.loc[year2, 'Content and Studio'] if 'Content and Studio' in hires_by_year_func.columns else 0

                    # Calculate percentage change, handling division by zero
                    prod_change = ((prod_y2 - prod_y1) / prod_y1 * 100) if prod_y1 > 0 else (np.inf if prod_y2 > 0 else 0)
                    cont_change = ((cont_y2 - cont_y1) / cont_y1 * 100) if cont_y1 > 0 else (np.inf if cont_y2 > 0 else 0)

                    # Format change strings
                    prod_change_str = f"{'+' if prod_change >= 0 else ''}{safe_round(prod_change, 0)}%" if np.isfinite(prod_change) else ("Increase from zero" if prod_y2 > 0 else "No change from zero")
                    cont_change_str = f"{'+' if cont_change >= 0 else ''}{safe_round(cont_change, 0)}%" if np.isfinite(cont_change) else ("Increase from zero" if cont_y2 > 0 else "No change from zero")

                    narratives['hiring_year_insight'] = (
                        f"Comparing hiring volume between {int(year1)} and {int(year2)}:"
                        f"<br><br>"
                        f"Product Development hiring changed by {prod_change_str}, while Content & Studio hiring changed by {cont_change_str}."
                    )
                else:
                    narratives['hiring_year_insight'] = "Insufficient historical data (less than 2 years with hiring activity) to compare year-over-year hiring trends."
            else:
                narratives['hiring_year_insight'] = "No valid numeric hire year data found for trend analysis."
        else:
             narratives['hiring_year_insight'] = "Hiring year insight requires 'hire_year' and 'job_function' columns with valid data."
    except Exception as e:
        print(f"Error generating hiring_year_insight: {e}")
        narratives['hiring_year_insight'] = "Error generating hiring year insight."

    # --- Hiring Level Insight (Top Levels by Function) ---
    try:
        if 'job_function' in comp_df.columns and 'job_level_title' in comp_df.columns:
            # Group hires by function and level
            hires_by_level_func = comp_df.groupby(['job_function', 'job_level_title'], observed=False).size().unstack(fill_value=0)
            prod_top_str = ""
            cont_top_str = ""

            # Get top 3 hiring levels for Product Development
            if 'Product Development' in hires_by_level_func.index:
                prod_levels = hires_by_level_func.loc['Product Development'].sort_values(ascending=False)
                prod_level_items = [f"<li>{lvl} ({val:,})</li>" for lvl, val in prod_levels.head(3).items() if val > 0 and lvl != 'Unknown']
                prod_top_str = f"<ul>{''.join(prod_level_items)}</ul>" if prod_level_items else "<li>No significant hiring volume found at specific levels.</li>"

            # Get top 3 hiring levels for Content & Studio
            if 'Content and Studio' in hires_by_level_func.index:
                cont_levels = hires_by_level_func.loc['Content and Studio'].sort_values(ascending=False)
                cont_level_items = [f"<li>{lvl} ({val:,})</li>" for lvl, val in cont_levels.head(3).items() if val > 0 and lvl != 'Unknown']
                cont_top_str = f"<ul>{''.join(cont_level_items)}</ul>" if cont_level_items else "<li>No significant hiring volume found at specific levels.</li>"

            narratives['hiring_level_insight'] = (
                f"The most common job levels for new hires appear to differ by function:"
                f"<br><b>Product Development Top Levels:</b>{prod_top_str}"
                f"<br><b>Content & Studio Top Levels:</b>{cont_top_str}"
            )
        else:
            narratives['hiring_level_insight'] = "Hiring level insight requires 'job_function' and 'job_level_title' columns with valid data."
    except Exception as e:
        print(f"Error generating hiring_level_insight: {e}")
        narratives['hiring_level_insight'] = "Error generating hiring level insight."

    # --- Combined Hiring Insight ---
    # Consolidates the year-over-year and level-based hiring insights.
    try:
        hiring_year_text = narratives.get('hiring_year_insight', "")
        hiring_level_text = narratives.get('hiring_level_insight', "")
        combined_hiring_text = ""

        # Add year text if valid
        if hiring_year_text and not hiring_year_text.startswith("Error") and not hiring_year_text.startswith("Insufficient") and not hiring_year_text.startswith("No valid"):
            combined_hiring_text += hiring_year_text

        # Add level text if valid, adding spacing
        if hiring_level_text and not hiring_level_text.startswith("Error"):
             if combined_hiring_text: # Add breaks if combining
                 combined_hiring_text += f"<br><br>Regarding the levels hired into, {hiring_level_text}"
             else:
                 combined_hiring_text = hiring_level_text # Use directly if it's the only text

        if combined_hiring_text:
            narratives['hiring_insight'] = combined_hiring_text
        else:
            narratives['hiring_insight'] = "Detailed hiring insights could not be generated due to missing or insufficient underlying data."

    except Exception as e:
        print(f"Error generating hiring_insight: {e}")
        narratives['hiring_insight'] = "Error combining hiring insights."


    # == Time to Promotion Section ==

    # --- Promotion Timeline Insight (Average Time by Level) ---
    ttp_by_level_data = pd.Series(dtype=float) # Initialize for storing data used later
    try:
        if 'time_to_promotion' in comp_df.columns and 'job_level_title' in comp_df.columns:
            # Filter for valid, numeric TTP data
            promo_timing_df = comp_df[comp_df['time_to_promotion'].notna()].copy()
            promo_timing_df['time_to_promotion'] = pd.to_numeric(promo_timing_df['time_to_promotion'], errors='coerce')
            promo_timing_df.dropna(subset=['time_to_promotion'], inplace=True)

            if not promo_timing_df.empty:
                # Group by level promoted *into* and calculate average TTP
                ttp_by_level_data = promo_timing_df.groupby('job_level_title')['time_to_promotion'].mean().round(1)
                # Exclude 'Unknown', sort by typical progression (can customize order)
                level_order_promo = ['Manager', 'Director', 'Vice President'] # Example order
                ttp_by_level_data = ttp_by_level_data.reindex(level_order_promo).dropna()
                narratives['ttp_by_level_data'] = ttp_by_level_data # Store data for strategic steps

                if not ttp_by_level_data.empty:
                    # Format as HTML list
                    ttp_items = [f"<li>{lvl}: {val} years</li>" for lvl, val in ttp_by_level_data.items()]
                    ttp_formatted_list = f"<ul>{''.join(ttp_items)}</ul>"
                    narratives['promotion_timeline_insight'] = (
                        f"The average time elapsed until promotion (Time to Promotion) varies by the level achieved:"
                        f"{ttp_formatted_list}"
                        f"This provides insight into typical progression timelines for these levels."
                    )
                else:
                    narratives['promotion_timeline_insight'] = "Could not calculate average time to promotion by relevant job levels after filtering."
                    narratives['ttp_by_level_data'] = pd.Series(dtype=float) # Ensure empty series if no data
            else:
                narratives['promotion_timeline_insight'] = "Insufficient valid numeric 'time_to_promotion' data to calculate by level."
                narratives['ttp_by_level_data'] = pd.Series(dtype=float)
        else:
            narratives['promotion_timeline_insight'] = "Promotion timeline insight requires 'time_to_promotion' and 'job_level_title' columns with valid data."
            narratives['ttp_by_level_data'] = pd.Series(dtype=float)
    except Exception as e:
        print(f"Error generating promotion_timeline_insight: {e}")
        narratives['promotion_timeline_insight'] = "Error generating promotion timeline insight."
        narratives['ttp_by_level_data'] = pd.Series(dtype=float) # Ensure empty series on error

    # --- Promotion Timing Summary (Copy for Template) ---
    # Some templates might use a different key for the same info, so copy if available.
    if 'promotion_timeline_insight' in narratives and not narratives['promotion_timeline_insight'].startswith("Error") and not narratives['promotion_timeline_insight'].startswith("Could not") and not narratives['promotion_timeline_insight'].startswith("Insufficient"):
        narratives['promotion_timing_summary'] = narratives['promotion_timeline_insight']
    else:
        # Ensure the key exists even if the original failed
        narratives['promotion_timing_summary'] = "Promotion timing summary (average TTP by level) could not be generated."

    # --- Promotion Velocity Insight (Rename from Regression) ---
    # Rename the key from the regression analysis for template compatibility.
    if 'promotion_regression_insight' in narratives:
        insight_text = narratives.pop('promotion_regression_insight') # Remove old key
        # Ensure appropriate line breaks if not already present
        if '<br>' not in insight_text and '.' in insight_text:
             insight_text = insight_text.replace('. ', '.<br><br>', 1)
        narratives['promo_velocity_insight'] = insight_text
    else:
        # Ensure key exists if regression failed
        narratives['promo_velocity_insight'] = "Promotion velocity insight (regression analysis) not available or encountered an error."

    # --- Tenure vs. Time to Promotion Correlation Insight ---
    try:
        if 'tenure_years' in comp_df.columns and 'time_to_promotion' in comp_df.columns:
            # Prepare data: numeric, non-null tenure and TTP
            ttp_tenure_df = comp_df[['tenure_years', 'time_to_promotion']].copy()
            ttp_tenure_df['tenure_years'] = pd.to_numeric(ttp_tenure_df['tenure_years'], errors='coerce')
            ttp_tenure_df['time_to_promotion'] = pd.to_numeric(ttp_tenure_df['time_to_promotion'], errors='coerce')
            ttp_tenure_df.dropna(inplace=True)

            if len(ttp_tenure_df) > 30: # Require sufficient data for meaningful correlation
                corr, p_val = pearsonr(ttp_tenure_df['tenure_years'], ttp_tenure_df['time_to_promotion'])

                # Describe correlation strength
                corr_desc = "weak"
                if abs(corr) > 0.7: corr_desc = "strong"
                elif abs(corr) > 0.4: corr_desc = "moderate"

                # Describe significance
                sig_text = f"(p = {p_val:.3f}, statistically significant)" if p_val < 0.05 else f"(p = {p_val:.3f}, not statistically significant)"
                direction = "positive" if corr > 0 else "negative" if corr < 0 else "negligible"

                # Interpretation based on direction
                interpretation = ""
                if corr > 0.1: # Threshold for positive interpretation
                     interpretation = "This suggests that employees with longer tenure at the time of their promotion also tended to have taken longer to reach that promotion (i.e., longer time between promotions or longer time since hire)."
                elif corr < -0.1: # Threshold for negative interpretation
                     interpretation = "This suggests that employees with longer tenure at the time of their promotion tended to have shorter times to promotion (i.e., faster progression)."
                else: # Weak or negligible correlation
                     interpretation = "This indicates little to no linear relationship between an employee's tenure and their time to promotion within this dataset."


                narratives['tenure_ttp_insight'] = (
                    f"Analysis reveals a {corr_desc}, {direction} correlation ({corr:.2f}) between employee tenure (at time of promotion) and their time-to-promotion {sig_text}."
                    f"<br><br>{interpretation}"
                )
            else:
                narratives['tenure_ttp_insight'] = "Insufficient data (<= 30 records with valid tenure and time-to-promotion) to reliably analyze the correlation between these two metrics."
        else:
            narratives['tenure_ttp_insight'] = "Correlation analysis requires 'tenure_years' and 'time_to_promotion' columns with valid numeric data."
    except Exception as e:
        print(f"Error generating tenure_ttp_insight: {e}")
        narratives['tenure_ttp_insight'] = "Error analyzing tenure vs. time-to-promotion correlation."


    # == Diagnostics Section ==

    # --- Data Quality Summary ---
    try:
        num_bad_dates = 0
        num_promo_issues = 0
        # Parse the JSON strings passed from load_data to get counts
        if bad_dates_json_str:
            try:
                bad_dates_list = json.loads(bad_dates_json_str)
                num_bad_dates = len(bad_dates_list)
            except json.JSONDecodeError:
                print("Warning: Could not parse bad_dates_json_str for quality summary.")
        if promo_issue_json_str:
            try:
                promo_issues_list = json.loads(promo_issue_json_str)
                num_promo_issues = len(promo_issues_list)
            except json.JSONDecodeError:
                print("Warning: Could not parse promo_issue_json_str for quality summary.")

        # Describe data processing steps
        processing_summary = (
            "<b>Data Processing Overview:</b><br>"
            "<ul>"
            "<li>Loaded employee snapshot and history data from CSV files.</li>"
            "<li>Standardized column names and converted key identifiers (employee, manager IDs) to appropriate types.</li>"
            "<li>Merged the snapshot and history datasets based on employee ID.</li>"
            "<li>Cleaned and standardized job function names (e.g., mapping 'Product' to 'Product Development').</li>"
            "<li>Parsed job level strings to extract numerical level and title components.</li>"
            "<li>Determined employee role (People Manager or Individual Contributor) based on job title keywords and reporting structure verification.</li>"
            "<li>Converted hire and promotion date strings to datetime objects, handling potential format errors.</li>"
            "<li>Calculated derived metrics: tenure, time-to-promotion (invalidating illogical dates), direct span of control, and total organizational scope (direct + indirect reports).</li>"
            "<li>Identified records with date format errors or promotion dates preceding hire dates for exclusion from relevant calculations and reporting in the diagnostics tab.</li>"
            "<li>Filtered the final dataset to include only records with valid, parseable hire dates for time-based analyses.</li>"
            "</ul>"
        )

        # Describe data quality findings based on counts
        quality_findings = "<br><b>Data Quality Findings:</b><br>"
        if num_bad_dates == 0 and num_promo_issues == 0:
            quality_findings += "No major date formatting errors or illogical promotion timing issues (promotion before hire) were detected in the source data based on the applied parsing rules."
        else:
            quality_findings += "Data quality checks identified the following potential issues:"
            findings_list_items = []
            if num_bad_dates > 0:
                findings_list_items.append(f"<li>{num_bad_dates} records contained date strings (hire or promotion) that could not be parsed into valid dates using the expected format. These records were excluded from time-based calculations where necessary.</li>")
            if num_promo_issues > 0:
                findings_list_items.append(f"<li>{num_promo_issues} records had a promotion date recorded earlier than the hire date. The time-to-promotion calculation for these records was invalidated.</li>")
            quality_findings += f"<ul>{''.join(findings_list_items)}</ul>"
            quality_findings += "Specific details for affected records are available in the Diagnostics tab."

        narratives['data_quality_summary'] = processing_summary + quality_findings

    except Exception as e:
        print(f"Error generating data_quality_summary: {e}")
        narratives['data_quality_summary'] = "Error generating data quality summary narrative."


    # == Strategic Section ==

    # --- Strategic Next Steps (Conditional Recommendations) ---
    try:
        strategic_next_steps_list = []
        # Retrieve necessary intermediate results
        diff_span_local = narratives.get('diff_span', np.nan)
        span_pval_local = narratives.get('span_p_value', np.nan)
        prod_span_local = narratives.get('prod_avg_span', np.nan)
        cont_span_local = narratives.get('cont_avg_span', np.nan)

        diff_tenure_local = narratives.get('diff_tenure', np.nan)
        tenure_pval_local = narratives.get('tenure_p_value', np.nan)
        prod_tenure_local = narratives.get('prod_avg_tenure', np.nan)
        cont_tenure_local = narratives.get('cont_avg_tenure', np.nan)

        ttp_by_level_local = narratives.get('ttp_by_level_data', pd.Series(dtype=float)) # Retrieve stored series

        # Condition 1: Significant and large span difference
        if not np.isnan(span_pval_local) and span_pval_local < 0.05 and not np.isnan(diff_span_local) and diff_span_local >= 1.0:
             strategic_next_steps_list.append(f"Investigate the impact of the significant difference in average manager span of control (Product: {safe_round(prod_span_local,1)} vs. Content: {safe_round(cont_span_local,1)}) on manager workload, team dynamics, and employee development opportunities.")

        # Condition 2: Significant and large tenure difference
        if not np.isnan(tenure_pval_local) and tenure_pval_local < 0.05 and not np.isnan(diff_tenure_local) and diff_tenure_local >= 0.5:
             strategic_next_steps_list.append(f"Explore the drivers behind the significant variation in average employee tenure (Product: {safe_round(prod_tenure_local, 1)} yrs vs. Content: {safe_round(cont_tenure_local, 1)} yrs), considering factors like retention patterns, role expectations, or career pathing within each function.")

        # Condition 3: Check TTP for specific levels (e.g., Director)
        if ttp_by_level_local is not None and not ttp_by_level_local.empty:
             if 'Director' in ttp_by_level_local.index:
                 dir_ttp = ttp_by_level_local.get('Director')
                 if dir_ttp is not None and not np.isnan(dir_ttp) and dir_ttp > 3.0: # Example threshold
                     strategic_next_steps_list.append(f"Review the promotion pathway to Director level, as the current average time-to-promotion ({dir_ttp:.1f} years) warrants examination regarding alignment with career progression expectations and potential barriers.")

        # Add standard exploratory steps
        strategic_next_steps_list.extend([
            "Analyze voluntary attrition rates, segmenting by function, level, and tenure, to identify potential retention challenges.",
            "If available, correlate manager span of control and total scope with team performance metrics or employee engagement survey results.",
            "Evaluate the effectiveness of different hiring sources (e.g., internal mobility vs. external hires) for key roles and leadership positions.",
            "Compare internal promotion rates against external hiring rates, particularly for senior and leadership levels, to assess internal talent pipeline health."
        ])

        # Format as a simple text list with newlines for the template
        # Jinja2 templates often handle markdown or simple text formatting better than raw HTML lists here.
        if strategic_next_steps_list:
             narratives['strategic_next_steps'] = "**Further Exploration Recommended:**\n" + "\n".join([f"- {step}" for step in strategic_next_steps_list])
        else:
             narratives['strategic_next_steps'] = "**Further Exploration Recommended:**\nNo specific areas were flagged based on the current analysis thresholds, but standard checks (e.g., attrition analysis, engagement correlation, hiring source review) remain valuable for ongoing organizational health monitoring."

    except Exception as e:
        print(f"Error generating strategic_next_steps: {e}")
        narratives['strategic_next_steps'] = "Error generating strategic recommendations."

    # --- Strategic Overall Context ---
    # Provides a high-level summary framing the detailed insights.
    try:
        # Use a standard framing narrative
        narratives['strategic_overall'] = (
             "Analysis of the Product Development and Content & Studio functions reveals distinct organizational patterns and talent dynamics. Variations exist in team structures (indicated by manager ratios and span of control), employee tenure profiles, and the velocity of talent movement (reflected in promotion rates and timelines)."
             "<br><br>"
             "These differences may stem from the unique operational requirements, strategic priorities, and historical evolution of each function. Understanding these variations is crucial for ensuring equitable employee experiences, optimizing organizational design, and supporting effective scaling and talent management strategies across diverse parts of the business."
             "<br><br>"
             "The detailed insights and recommended next steps aim to facilitate deeper investigation into areas where observed differences may warrant further attention or strategic intervention."
        )
    except Exception as e:
        print(f"Error generating strategic_overall: {e}")
        narratives['strategic_overall'] = "Error generating overall strategic summary."

    # --- Strategic Insight 1: Structure Reflects Strategy ---
    # Summarizes findings related to org structure (manager ratio, span).
    try:
        org_design_text = narratives.get('org_design_insight', '')
        span_text = narratives.get('span_control_insight', '')
        insight_parts = []
        if org_design_text and not org_design_text.startswith("Error"): insight_parts.append(org_design_text)
        if span_text and not span_text.startswith("Error") and not span_text.startswith("Detailed span"): insight_parts.append(span_text)

        if insight_parts:
             narratives['strategic_insight_1'] = "<b>Organizational structures appear tailored, potentially reflecting differing strategic needs and operational models:</b><br><br>" + "<br><br>".join(insight_parts)
        else:
             narratives['strategic_insight_1'] = "<b>Organizational Structure Insight:</b><br>Analysis of manager ratios and span of control could not be completed due to data limitations."
    except Exception as e:
        print(f"Error generating strategic_insight_1: {e}")
        narratives['strategic_insight_1'] = "Error generating strategic insight on structure."

    # --- Strategic Insight 2: Differing Talent Velocities ---
    # Repurposes the executive summary findings about significant differences.
    if 'exec_summary_findings' in narratives:
        findings_text = narratives.pop('exec_summary_findings') # Remove old key
        if findings_text and not findings_text.startswith("Error") and not findings_text.startswith("No statistically significant"):
            narratives['strategic_insight_2'] = f"<b>Key differences in talent velocity and structure observed:</b><br>{findings_text}"
        else:
            # Provide specific fallback if findings were non-significant or errored
            narratives['strategic_insight_2'] = "<b>Talent Velocity Comparison:</b><br>Analysis did not identify statistically significant differences meeting thresholds in tenure, span, promotion rates, or controlled promotion timing between the functions."
    else:
        # Ensure key exists if base findings failed earlier
         narratives['strategic_insight_2'] = "<b>Talent Velocity Comparison:</b><br>Comparative analysis of talent velocity metrics encountered an error or data was insufficient."


    # --- Strategic Insight 3: Operational Cycles Drive Talent Flow ---
    # Summarizes findings related to seasonality (promotions, hiring trends).
    try:
        promo_season_text = narratives.get('promotion_seasonality_insight', '')
        hiring_year_text = narratives.get('hiring_year_insight', '')
        insight_parts = []
        if promo_season_text and not promo_season_text.startswith("Error") and not promo_season_text.startswith("Insufficient"): insight_parts.append(promo_season_text)
        if hiring_year_text and not hiring_year_text.startswith("Error") and not hiring_year_text.startswith("Insufficient"): insight_parts.append(hiring_year_text)

        if insight_parts:
            narratives['strategic_insight_3'] = "<b>Talent movement shows seasonality, potentially linked to operational cycles:</b><br><br>" + "<br><br>".join(insight_parts)
        else:
            narratives['strategic_insight_3'] = "<b>Operational Cycles Insight:</b><br>Analysis of seasonality in promotions and hiring trends could not be completed due to data limitations."
    except Exception as e:
        print(f"Error generating strategic_insight_3: {e}")
        narratives['strategic_insight_3'] = "Error generating strategic insight on operational cycles."

    # --- Strategic ML Insight (Informational Placeholder) ---
    # Adds context about potential future analyses.
    narratives['strategic_ml_insight'] = "Advanced analytical techniques, such as predictive modeling for attrition risk or promotion likelihood using machine learning, could offer deeper strategic insights. These methods can help identify subtle patterns and leading indicators not apparent through descriptive statistics alone but are beyond the scope of this current dashboard version."

    # --- Add Executive Summary Components (for top cards) ---
    # These provide quick numerical summaries.
    try:
        total_employees = narratives.get('total_employees', len(comp_df)) # Use intermediate if available
        role_counts = comp_df['role'].value_counts() if 'role' in comp_df.columns else pd.Series(dtype=int)
        level_counts = comp_df['job_level_title'].value_counts() if 'job_level_title' in comp_df.columns else pd.Series(dtype=int)
        func_total = comp_df['job_function'].value_counts() if 'job_function' in comp_df.columns else pd.Series(dtype=int)
        prod_total_exec = func_total.get('Product Development', 0)
        cont_total_exec = func_total.get('Content and Studio', 0)

        # Retrieve calculated averages/medians from intermediate results
        avg_span_exec = narratives.get('avg_span', np.nan)
        prod_span_exec = narratives.get('prod_avg_span', np.nan)
        cont_span_exec = narratives.get('cont_avg_span', np.nan)
        avg_tenure_exec = narratives.get('avg_tenure', np.nan)
        prod_tenure_exec = narratives.get('prod_avg_tenure', np.nan)
        cont_tenure_exec = narratives.get('cont_avg_tenure', np.nan)

        # Format executive summary strings
        narratives['exec_summary_employees'] = (
            f"Total Analyzed: {total_employees:,} (Prod: {prod_total_exec:,}, Content: {cont_total_exec:,})"
            f"<br>Role Split: ICs={role_counts.get('Individual Contributor', 0):,}, Managers={role_counts.get('People Manager', 0):,}"
            f"<br>Top Levels: {', '.join([f'{lvl} ({cnt:,})' for lvl, cnt in level_counts.head(3).items()])}"
        )
        narratives['exec_summary_span'] = (
             f"Avg Span (Overall): {safe_round(avg_span_exec, 1)}"
             f"<br>Avg Span (Prod): {safe_round(prod_span_exec, 1)}"
             f"<br>Avg Span (Content): {safe_round(cont_span_exec, 1)}"
        )
        narratives['exec_summary_tenure'] = (
            f"Avg Tenure (Overall): {safe_round(avg_tenure_exec, 1)} yrs"
            f"<br>Avg Tenure (Prod): {safe_round(prod_tenure_exec, 1)} yrs"
            f"<br>Avg Tenure (Content): {safe_round(cont_tenure_exec, 1)} yrs"
        )
    except Exception as e:
        print(f"Error generating exec_summary card narratives: {e}")
        narratives['exec_summary_employees'] = "Error generating employee breakdown."
        narratives['exec_summary_tenure'] = "Error generating tenure summary."
        narratives['exec_summary_span'] = "Error generating span summary."


    # --- Final Cleanup ---
    # Remove intermediate calculation results and temporary keys before returning
    intermediate_keys_to_remove = {
        'total_employees', 'comp_by_func_role', 'ic_pct', 'mgr_pct', 'prod_mgr_pct',
        'cont_mgr_pct', 'roles_p_value', 'roles_stat_sig_text',
        'avg_tenure', 'prod_avg_tenure', 'cont_avg_tenure', 'prod_pct_new',
        'cont_pct_new', 'diff_tenure', 'tenure_p_value', 'tenure_stat_sig_text',
        'prod_mgr_ratio', 'cont_mgr_ratio',
        'avg_span', 'prod_avg_span', 'cont_avg_span', 'diff_span', 'span_p_value', 'span_stat_sig_text',
        'promo_counts', 'total_by_func', 'promo_rate_prod', 'promo_rate_cont',
        'promo_rate_p_value', 'promo_rate_stat_sig_text',
        'ttp_regression_model', 'ttp_regression_beta', 'ttp_regression_pval',
        'ttp_by_level_data' # Remove the stored series data
    }

    # Remove the identified keys
    for key in intermediate_keys_to_remove:
        narratives.pop(key, None) # Use pop with default to avoid errors if key missing

    # Final check for any remaining error messages and replace with standard fallback
    final_keys = list(narratives.keys())
    for key in final_keys:
        value = narratives.get(key)
        if isinstance(value, str) and (value.startswith("Error generating") or value.startswith("Could not calculate") or value.endswith("unavailable.") or value.startswith("Insufficient data") or value.startswith("Note:")):
            # Replace specific error/note messages with a generic indicator for the dashboard
            narratives[key] = f"Insight generation for '{key}' encountered an issue or data was unavailable/insufficient for analysis."

    return narratives


# =============================================================================
# # 6. Chart Creation
# =============================================================================
# Description: This section defines the `create_charts` function, which
# generates a variety of visualizations using Plotly Express and Plotly
# Graph Objects. It takes the processed DataFrame as input and returns a
# dictionary of chart specifications suitable for JSON serialization and
# embedding into the HTML template. An inner helper function `encode_chart`
# standardizes styling and encoding.

# Define safe_round here if it wasn't defined globally or imported properly
# For context, assuming it's available from Part 1.
def safe_round(value, decimals=1, na_value='N/A'):
    """
    Safely rounds a numerical value to a specified number of decimal places.

    Handles non-numeric inputs (like NaN, None, or strings) by returning a
    specified placeholder value ('N/A' by default).

    Args:
        value (any): The value to round.
        decimals (int): The number of decimal places to round to. Defaults to 1.
        na_value (str): The value to return if the input is not a valid number.
                        Defaults to 'N/A'.

    Returns:
        float or str: The rounded number, or the na_value placeholder.
    """
    if pd.isna(value) or not isinstance(value, (int, float, np.number)):
        return na_value
    try:
        # Use np.round for compatibility with numpy types
        return np.round(value, decimals)
    except (TypeError, ValueError):
         # Catch potential errors during rounding itself
         return na_value


# --- Chart Creation Function ---
def create_charts(df):
    """
    Generates various Plotly charts from the input DataFrame.

    Creates visualizations covering workforce composition, tenure distribution,
    span of control, promotion trends, hiring patterns, and time-to-promotion
    metrics. Applies consistent styling and formatting using an inner helper.

    Args:
        df (pd.DataFrame): The processed DataFrame from `load_data`.

    Returns:
        dict: A dictionary where keys are chart identifiers (matching template
              expectations) and values are Plotly figure specifications encoded
              as Python dictionaries. Returns an empty dict if input is invalid.
    """
    charts = {}
    if df is None or df.empty:
        print("Input DataFrame is empty or None. No charts will be generated.")
        return charts

    # --- Configuration & Setup ---
    # Access global color palette (defined in Part 1)
    global color_palette
    if 'color_palette' not in globals():
        print("Warning: color_palette not defined globally. Using default colors.")
        # Define fallback colors if global palette is missing
        color_palette = {'primary': '#E50914', 'secondary': '#B3B3B3', 'tertiary': '#808080'}

    # Define primary colors for direct use
    netflix_red = color_palette.get('primary', '#E50914')
    netflix_grey = color_palette.get('secondary', '#B3B3B3')
    netflix_other = color_palette.get('tertiary', '#808080')

    # Map job functions to specific colors for consistency across charts
    color_map = {
        'Product Development': netflix_red,
        'Content and Studio': netflix_grey,
        'Other': netflix_other
    }

    # Check for required columns needed for various charts
    required_cols = ['job_function', 'job_level_title', 'role', 'tenure_years',
                     'is_manager', 'span_of_control', 'is_promoted', 'promotion_year',
                     'hire_quarter', 'time_to_promotion', 'hire_year', 'promotion_date_dt', # Use datetime version
                     'hire_date_dt'] # Use datetime version
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        print(f"Warning: Input DataFrame is missing required columns for charts: {', '.join(missing_cols)}. Some charts may fail or be skipped.")

    # Filter data for primary functions, falling back if necessary
    comp_df = df[df['job_function'].isin(['Product Development', 'Content and Studio'])].copy()
    if comp_df.empty:
        print("Warning: No data found for 'Product Development' or 'Content and Studio'. Using all data for charts.")
        comp_df = df.copy()

    if comp_df.empty:
        print("Filtered or original DataFrame is empty. No charts generated.")
        return charts

    # Define standard order for job levels for consistent axis sorting
    level_order = ['Individual Contributor', 'Manager', 'Director', 'Vice President', 'Unknown']
    manager_level_order = ['Manager', 'Director', 'Vice President'] # Order for manager-specific charts

    # --- Inner Helper Function to Encode Charts ---
    def encode_chart(fig, chart_title=""):
        """
        Applies standard layout updates and encodes a Plotly figure to dict.

        Handles styling for common trace types (bar, pie, scatter, box, etc.),
        applies base layout defaults (margins, legend position, background),
        and performs conditional layout adjustments based on chart type. Uses
        `pio.to_json` followed by `json.loads` for robust serialization.

        Args:
            fig (go.Figure): The Plotly figure object.
            chart_title (str): The title for the chart (used for logging/context).

        Returns:
            dict or None: The figure specification as a dictionary, or None if encoding fails.
        """
        if not isinstance(fig, go.Figure):
            print(f"Warning: Attempted to encode non-figure object for chart '{chart_title}'")
            return None

        # --- Base Layout Defaults (Applied to all charts) ---
        base_layout = {
            'margin': dict(l=60, r=150, t=60, b=100), # Margins to accommodate labels/legends
            'legend': dict(
                orientation='v', yanchor='middle', y=0.5, # Vertical legend on the right
                xanchor='left', x=1.02,
                bgcolor='rgba(30,30,30,0.7)', bordercolor='rgba(255,255,255,0.2)', borderwidth=1,
                font=dict(size=10, color='#DDDDDD')
            ),
            'title': {'text': chart_title, 'x': 0.5, 'xanchor': 'center', 'font': {'size': 16}},
            'xaxis': {
                'automargin': True, 'tickangle': -45, 'showticklabels': True, # Default angle for x-axis labels
                'title_standoff': 20, 'title_font_size': 12, 'tickfont_size': 10,
                'gridcolor': 'rgba(128,128,128,0.2)', # Subtle grid lines
                'linecolor': 'rgba(128,128,128,0.5)',
                'zerolinecolor': 'rgba(128,128,128,0.3)',
            },
            'yaxis': {
                'automargin': True, 'title_standoff': 15,
                'title_font_size': 12, 'tickfont_size': 10,
                'gridcolor': 'rgba(128,128,128,0.2)', # Subtle grid lines
                'linecolor': 'rgba(128,128,128,0.5)',
                'zerolinecolor': 'rgba(128,128,128,0.3)',
            },
            'paper_bgcolor': 'rgba(0,0,0,0)', # Transparent background
            'plot_bgcolor': 'rgba(0,0,0,0)',  # Transparent plot area
            'hovermode': 'closest',
            'hoverlabel': dict( # Styling for hover tooltips
                bgcolor="rgba(15, 15, 15, 0.9)", font_size=12,
                font_family="Arial, sans-serif", font_color="#FFFFFF",
                bordercolor="rgba(255,255,255,0.4)"
            )
            # Font color is inherited from the global template setting
        }

        # --- Apply Trace Styling (Based on trace type) ---
        for trace in fig.data:
            try:
                trace_type = getattr(trace, 'type', None)
                # Common styling: marker lines for clarity
                if hasattr(trace, 'marker') and not hasattr(trace.marker, 'line'):
                    trace.marker.line = {}
                if hasattr(trace, 'marker') and hasattr(trace.marker, 'line'):
                     trace.marker.line.width = getattr(trace.marker.line, 'width', 0.5)
                     trace.marker.line.color = getattr(trace.marker.line, 'color', 'rgba(255, 255, 255, 0.3)')

                # Type-specific styling and hovertemplates
                if trace_type == 'bar':
                    trace.marker.opacity = getattr(trace.marker, 'opacity', 0.85)
                    if not getattr(trace, 'hovertemplate', None):
                        trace.hovertemplate = '<b>%{data.name}</b><br>%{x}: %{y}<extra></extra>'
                elif trace_type == 'pie':
                    # trace.marker.opacity = getattr(trace.marker, 'opacity', 0.9) # Opacity on marker causes error
                    trace.opacity = getattr(trace, 'opacity', 0.9) # Apply opacity to the trace itself
                    trace.marker.line.width = getattr(trace.marker.line, 'width', 1) # Slightly thicker line for pies
                    if not getattr(trace, 'hovertemplate', None):
                        trace.hovertemplate = '<b>%{data.name}</b><br>%{label}: %{percent:.1%}<extra></extra>'
                elif trace_type == 'scatter':
                    if hasattr(trace, 'mode') and ('lines' in trace.mode):
                        trace.line = getattr(trace, 'line', {})
                        trace.line.width = getattr(trace.line, 'width', 2.5)
                        trace.opacity = getattr(trace, 'opacity', 0.8)
                    if hasattr(trace, 'mode') and ('markers' in trace.mode):
                        trace.marker = getattr(trace, 'marker', {})
                        trace.marker.opacity = getattr(trace.marker, 'opacity', 0.7)
                        trace.marker.size=getattr(trace.marker, 'size', 6)
                    if not getattr(trace, 'hovertemplate', None):
                         # Default hover for scatter, customize per chart if needed
                         trace.hovertemplate = '<b>%{data.name}</b><br>X: %{x}<br>Y: %{y}<extra></extra>'
                elif trace_type == 'box':
                    # Apply line style to box plots for consistency
                    trace.marker.line.width = 1.5
                    trace.marker.line.color = 'rgba(255, 255, 255, 0.4)'
                    # Simplified hovertemplate for box plots
                    trace.hovertemplate = ('<b>%{x}</b> (%{data.name})<br>Median: %{median:.1f}<br>Q1: %{q1:.1f} | Q3: %{q3:.1f}<extra></extra>')
                elif trace_type == 'histogram':
                    trace.marker.opacity = getattr(trace.marker, 'opacity', 0.7)
                    if not getattr(trace, 'hovertemplate', None):
                        trace.hovertemplate = '<b>%{data.name}</b><br>Range: %{x}<br>Count: %{y}<extra></extra>'
                elif trace_type == 'indicator':
                    if hasattr(trace, 'gauge'): trace.gauge.borderwidth = getattr(trace.gauge, 'borderwidth', 0)
                    # Hover info typically not needed/used for indicators
                elif trace_type == 'treemap':
                     trace.marker.line.width = 1
                     trace.marker.line.color = 'rgba(0,0,0,0.4)' # Darker line for contrast on light colors
                     trace.textfont = getattr(trace, 'textfont', {}); trace.textfont.size = 11; trace.textfont.color = '#FFFFFF'
                     # Default hovertemplate for treemaps
                     trace.hovertemplate = '<b>%{label}</b><br>Value: %{value}<br>Parent: %{parent}<extra></extra>'

            except AttributeError as ae: print(f"Warning: Attribute error styling trace for '{chart_title}': {ae}")
            except Exception as e_trace: print(f"Warning: General error styling trace for '{chart_title}': {e_trace}")

        # --- Apply Layout Updates ---
        fig.update_layout(base_layout) # Apply base layout first

        # --- Conditional Layout Adjustments ---
        # Force category axis for specific time-based charts if needed
        if chart_title in ['Promotions Over Time', 'Annual Hiring Volume', 'Approximate Annual Promotion Rate (%)', 'Hiring by Quarter (Trend)']:
            if fig.layout.xaxis: fig.update_xaxes(type='category')

        # Specific layout adjustments for specific chart types
        if fig.data and hasattr(fig.data[0], 'type'):
            first_trace_type = fig.data[0].type
            if first_trace_type == 'sunburst': # Example: Sunburst often needs minimal margins
                fig.update_layout(margin=dict(l=10, r=10, t=30, b=10), showlegend=False)
            elif first_trace_type == 'pie': # Example: Donut charts might need specific legend handling
                 # Handled per-chart basis if needed (like 'composition' chart)
                 pass
            elif first_trace_type == 'indicator': # Gauges often need different margins
                 fig.update_layout(margin=dict(l=20, r=20, t=50, b=20))

        # Adjustments for faceted charts (e.g., promotion quarters)
        if chart_title == 'Promotions by Quarter (Faceted by Year)':
             if fig.layout.annotations: # Clean facet labels
                  fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
             # Ensure consistent axis titles across facets if needed
             # fig.update_xaxes(title_text="Quarter", matches='x') # Example if needed
             # fig.update_yaxes(title_text="# Promotions", matches='y') # Example if needed

        # Adjustments for Distplot (from figure_factory)
        if chart_title == 'Distribution of Time to Promotion (Distplot)':
             fig.update_layout(barmode='overlay') # Overlay histograms
             for trace in fig.data: # Style histogram and curve traces
                 if trace.type == 'histogram': trace.marker.opacity = 0.6
                 elif trace.type == 'scatter' and trace.mode == 'lines': trace.line.width = 2.5

        # --- Encode to Dictionary ---
        try:
            # Use pio.to_json for robust serialization (handles numpy types), then load back to dict
            return json.loads(pio.to_json(fig))
        except Exception as e:
            print(f"Error encoding chart '{chart_title}' using pio.to_json/json.loads: {e}")
            return None
    # --- End of encode_chart ---


    # --- Chart Generation Blocks ---
    # Each block attempts to create a specific chart, using the processed data
    # and the encode_chart helper for styling and serialization.

    # Chart 1: Composition by Job Level ('composition' / 'composition_detail') - Donut Charts
    try:
        fig = None
        comp_data = comp_df.groupby(['job_function', 'job_level_title'], observed=False).size().reset_index(name='count')
        comp_data['job_level_title'] = pd.Categorical(comp_data['job_level_title'], categories=level_order, ordered=True)
        comp_data = comp_data.sort_values(['job_function', 'job_level_title'])

        if not comp_data.empty:
            fig = go.Figure()
            functions = comp_data['job_function'].unique()
            level_name_map = {'Individual Contributor': 'IC', 'Manager': 'Manager', 'Director': 'Director', 'Vice President': 'VP'}

            # Create Product Development Donut
            if 'Product Development' in functions:
                prod_data = comp_data[comp_data['job_function'] == 'Product Development']
                prod_total = prod_data['count'].sum()
                prod_data['percentage'] = prod_data['count'] / prod_total * 100
                prod_data['level_short'] = prod_data['job_level_title'].map(level_name_map).fillna(prod_data['job_level_title'])
                prod_data['legend_text'] = prod_data.apply(lambda row: f"{row['level_short']}: {int(row['count'])} ({row['percentage']:.1f}%)", axis=1)
                fig.add_trace(go.Pie(
                    values=prod_data['count'], labels=prod_data['legend_text'], name='Product Development',
                    domain=dict(x=[0, 0.40], y=[0, 1]), hole=0.6,
                    marker_colors=[netflix_red, '#FF4D4D', '#FF8080', '#FFB3B3'], # Shades of primary
                    textfont=dict(color='white', size=11), textinfo='none', showlegend=True, legendgroup="Prod"
                ))
                fig.add_annotation(x=0.20, y=0.5, text="Product<br>Development", showarrow=False, font=dict(size=13))

            # Create Content & Studio Donut
            if 'Content and Studio' in functions:
                content_data = comp_data[comp_data['job_function'] == 'Content and Studio']
                content_total = content_data['count'].sum()
                content_data['percentage'] = content_data['count'] / content_total * 100
                content_data['level_short'] = content_data['job_level_title'].map(level_name_map).fillna(content_data['job_level_title'])
                content_data['legend_text'] = content_data.apply(lambda row: f"{row['level_short']}: {int(row['count'])} ({row['percentage']:.1f}%)", axis=1)
                fig.add_trace(go.Pie(
                    values=content_data['count'], labels=content_data['legend_text'], name='Content and Studio',
                    domain=dict(x=[0.43, 0.83]), hole=0.6,
                    marker_colors=['#8E8E8E', '#AAAAAA', '#CCCCCC', '#DDDDDD'], # Shades of grey
                    textfont=dict(color='white', size=11), textinfo='none', showlegend=True, legendgroup="Cont"
                ))
                fig.add_annotation(x=0.63, y=0.5, text="Content<br>& Studio", showarrow=False, font=dict(size=13))

            # Update layout for donut specifics (legend on right, height)
            fig.update_layout(
                height=380, showlegend=True,
                legend=dict(orientation="v", yanchor="middle", y=0.5, xanchor="left", x=0.85, itemsizing="constant")
                # Base layout handles margins, bg color etc.
            )

            # Encode the chart
            encoded_chart = encode_chart(fig, 'Composition by Job Level')
            if encoded_chart:
                charts['composition'] = encoded_chart
                charts['composition_detail'] = encoded_chart # Use same chart for both keys
            else:
                print("Skipping 'composition': Encoding failed.")
        else:
            print("Skipping 'composition': No data for donut charts.")
    except Exception as e:
        print(f"Error generating Chart 'composition' (Donut): {e}")
        traceback.print_exc()
        if 'composition' in charts: del charts['composition']
        if 'composition_detail' in charts: del charts['composition_detail']

    # Chart 2: Roles (Manager vs IC) (Vertical Bar)
    try:
        fig = None
        if 'role' in comp_df.columns:
            role_data = comp_df.groupby(['job_function', 'role'], observed=False).size().reset_index(name='count')
            fig = px.bar(role_data, x='role', y='count', color='job_function',
                         barmode='group', color_discrete_map=color_map,
                         labels={'role': 'Role', 'count': '# Employees', 'job_function': 'Function'})
            if fig:
                 encoded_chart = encode_chart(fig, 'Managers vs. ICs')
                 if encoded_chart: charts['roles'] = encoded_chart
                 else: print("Skipping 'roles': Encoding failed.")
            else:
                 print("Skipping 'roles': Figure object not created.")
        else:
            print("Skipping 'roles': Missing 'role' column.")
    except Exception as e:
        print(f"Error generating Chart 'roles': {e}")
        traceback.print_exc()
        if 'roles' in charts: del charts['roles']

    # Chart 3: Tenure Distribution (ECDF) ('tenure_distribution_violin') - ECDF Line
    try:
        fig = None
        if 'tenure_years' in comp_df.columns and comp_df['tenure_years'].notna().any():
            fig = go.Figure()
            for function in sorted(comp_df['job_function'].unique()):
                # Ensure tenure is numeric and handle NaNs
                func_data = pd.to_numeric(comp_df[comp_df['job_function'] == function]['tenure_years'], errors='coerce').dropna()
                if not func_data.empty:
                    ecdf = sm.distributions.ECDF(func_data)
                    x_min, x_max = func_data.min(), func_data.max()
                    # Handle case where all values are the same
                    x_vals = np.array([x_min]) if x_min == x_max else np.linspace(x_min, x_max, num=100)
                    y_vals = ecdf(x_vals)
                    color = color_map.get(function, netflix_other)
                    fig.add_trace(go.Scatter(
                        x=x_vals, y=y_vals, mode='lines', name=function, line=dict(color=color),
                        hovertemplate=(f'<b>{function}</b><br>Tenure: %{{x:.1f}} Yrs<br>Cum. Prob: %{{y:.2f}}<extra></extra>')
                    ))
            fig.update_layout(xaxis_title='Tenure (Yrs)', yaxis_title='Cumulative Probability')
            if fig and fig.data:
                encoded_chart = encode_chart(fig, 'Tenure Distribution (ECDF)')
                if encoded_chart: charts['tenure_distribution_violin'] = encoded_chart
                else: print("Skipping 'tenure_distribution_violin': Encoding failed.")
            else: print("Skipping 'tenure_distribution_violin': Figure object created but has no data.")
        else: print("Skipping 'tenure_distribution_violin': Missing or insufficient 'tenure_years' data.")
    except ImportError: print("Error generating Chart 'tenure_distribution_violin': statsmodels not installed or found.")
    except Exception as e:
        print(f"Error generating Chart 'tenure_distribution_violin': {e}"); traceback.print_exc()
        if 'tenure_distribution_violin' in charts: del charts['tenure_distribution_violin']

    # Chart 4: Span of Control (Box Plot) ('span_control_box')
    try:
        fig = None
        if 'is_manager' in comp_df.columns and 'span_of_control' in comp_df.columns:
            managers_df = comp_df[comp_df['is_manager']].copy()
            managers_df['span_of_control'] = pd.to_numeric(managers_df['span_of_control'], errors='coerce')
            managers_df_filtered = managers_df[managers_df['span_of_control'] > 0]

            if not managers_df_filtered.empty:
                fig = px.box(
                    managers_df_filtered,
                    y='span_of_control',
                    x='job_function',
                    color='job_function',
                    points='outliers',
                    color_discrete_map=color_map,
                    labels={
                        'span_of_control': 'Direct Reports (Span>0)',
                        'job_function': 'Function'
                    }
                )

                for trace in fig.data:
                    func = trace.name
                    trace.update(
                        hoverinfo='text',
                        hovertemplate=None,
                        text=[f"Function: {func}<br>Direct Reports: {trace.y[i]:.0f}" for i in range(len(trace.y))],
                        hoverlabel=dict(bgcolor='rgba(0,0,0,0.9)', font_size=12, font_family='Arial')
                    )

                encoded_chart = encode_chart(fig, 'Span of Control (Managers w/ Reports)')
                if encoded_chart:
                    charts['span_control_box'] = encoded_chart
                else:
                    print("Skipping 'span_control_box': Encoding failed.")
            else:
                print("Skipping 'span_control_box': No managers found with > 0 reports.")
        else:
            print("Skipping 'span_control_box': Missing 'span_of_control' or 'is_manager' data.")
    except Exception as e:
        print(f"Error generating Chart 'span_control_box': {e}")
        traceback.print_exc()
        charts.pop('span_control_box', None)

    # Chart 5: Promotions Trend ('promotions_trend') - Line
    try:
        fig = None
        if 'is_promoted' in comp_df.columns and 'promotion_year' in comp_df.columns:
            promo_df = comp_df[comp_df['is_promoted']].copy()
            # Ensure promotion_year is numeric and handle NaNs
            promo_df['promotion_year'] = pd.to_numeric(promo_df['promotion_year'], errors='coerce').astype('Int64')
            promo_df.dropna(subset=['promotion_year'], inplace=True)

            if not promo_df.empty:
                promo_trend_data = promo_df.groupby(['promotion_year', 'job_function'], observed=False).size().reset_index(name='count')
                fig = px.line(promo_trend_data, x='promotion_year', y='count', color='job_function',
                              markers=True, color_discrete_map=color_map,
                              labels={'count': '# Promotions', 'promotion_year': 'Year', 'job_function': 'Function'})
                if fig:
                    fig.update_traces(hovertemplate='<b>%{data.name}</b><br>Year: %{x}<br>Promotions: %{y}<extra></extra>')
                    encoded_chart = encode_chart(fig, 'Promotions Over Time') # encode_chart handles category axis
                    if encoded_chart: charts['promotions_trend'] = encoded_chart
                    else: print("Skipping 'promotions_trend': Encoding failed.")
                else: print("Skipping 'promotions_trend': Figure object not created.")
            else: print("Skipping 'promotions_trend': No valid numeric promotion years found after cleaning.")
        else: print("Skipping 'promotions_trend': Missing 'promotion_year' or 'is_promoted' data.")
    except Exception as e:
        print(f"Error generating Chart 'promotions_trend': {e}"); traceback.print_exc()
        if 'promotions_trend' in charts: del charts['promotions_trend']

    # Chart 6: Hiring by Quarter ('hiring_quarter') - Line
    try:
        fig = None
        # Requires hire_year (numeric) and hire_quarter (numeric/string)
        if 'hire_year' in comp_df.columns and 'hire_quarter' in comp_df.columns:
             # Ensure hire_year is numeric Int64
             comp_df['hire_year_num'] = pd.to_numeric(comp_df['hire_year'], errors='coerce').astype('Int64')
             # Ensure hire_quarter is usable (e.g., string 'YYYYQ#')
             comp_df['hire_year_quarter_str'] = np.where(
                  comp_df['hire_year_num'].notna() & comp_df['hire_quarter'].notna(),
                  comp_df['hire_year_num'].astype(str) + 'Q' + comp_df['hire_quarter'].astype(str).str.replace(r'\.0$', '', regex=True), # Handle potential float conversion
                  pd.NA
             )
             hire_q_data = comp_df.dropna(subset=['hire_year_quarter_str']).groupby(
                 ['hire_year_quarter_str', 'job_function'], observed=False
             ).size().reset_index(name='count')
             # Sort by the constructed year-quarter string for chronological order
             hire_q_data = hire_q_data.sort_values('hire_year_quarter_str')

             if not hire_q_data.empty:
                 fig = px.line(hire_q_data, x='hire_year_quarter_str', y='count', color='job_function',
                              markers=True, color_discrete_map=color_map,
                              labels={'count': '# Hires', 'hire_year_quarter_str': 'Year-Quarter', 'job_function': 'Function'})
                 if fig:
                     fig.update_traces(hovertemplate='<b>%{data.name}</b><br>Quarter: %{x}<br>Hires: %{y}<extra></extra>')
                     encoded_chart = encode_chart(fig, 'Hiring by Quarter (Trend)') # encode_chart handles category axis
                     if encoded_chart: charts['hiring_quarter'] = encoded_chart
                     else: print("Skipping 'hiring_quarter': Encoding failed.")
                 else: print("Skipping 'hiring_quarter': Figure object not created.")
             else: print("Skipping 'hiring_quarter': No data after creating YYYYQ# labels.")
        else: print("Skipping 'hiring_quarter': Missing 'hire_quarter' or 'hire_year' data.")
    except Exception as e:
        print(f"Error generating Chart 'hiring_quarter' (Line): {e}"); traceback.print_exc()
        if 'hiring_quarter' in charts: del charts['hiring_quarter']

    # Chart 7: Time to Promotion by Level ('promo_timing_level' / 'promo_timing_level_main') - Box
    try:
        fig = None
        promo_df_ttp = pd.DataFrame()
        if 'time_to_promotion' in comp_df.columns:
            comp_df['time_to_promotion_num'] = pd.to_numeric(comp_df['time_to_promotion'], errors='coerce')
            promo_df_ttp = comp_df[comp_df['time_to_promotion_num'] >= 0].copy()

        if not promo_df_ttp.empty and 'job_level_title' in promo_df_ttp.columns:
            promo_df_ttp['job_level_title'] = pd.Categorical(promo_df_ttp['job_level_title'], categories=level_order, ordered=True)
            promo_df_ttp = promo_df_ttp.sort_values('job_level_title')

            fig = px.box(
                promo_df_ttp,
                x='job_level_title',
                y='time_to_promotion_num',
                color='job_function',
                points=False,
                color_discrete_map=color_map,
                labels={
                    'job_level_title': 'Level Promoted Into',
                    'time_to_promotion_num': 'Time to Promotion (Yrs)',
                    'job_function': 'Function'
                }
            )

            for trace in fig.data:
                job_func = trace.name
                trace.update(
                    hoverinfo='text',
                    hovertemplate=None,
                    text=[f"Level: {trace.x[i]}<br>Function: {job_func}" for i in range(len(trace.x))],
                    hoverlabel=dict(bgcolor='rgba(0,0,0,0.9)', font_size=12, font_family='Arial')
                )

            encoded_chart = encode_chart(fig, 'Time to Promotion by Level')
            if encoded_chart:
                charts['promo_timing_level'] = encoded_chart
                charts['promo_timing_level_main'] = encoded_chart
            else:
                print("Skipping 'promo_timing_level': Encoding failed.")
        else:
            print("Skipping 'promo_timing_level': Missing/invalid 'time_to_promotion' or 'job_level_title' data.")
    except Exception as e:
        print(f"Error generating Chart 'promo_timing_level': {e}")
        traceback.print_exc()
        charts.pop('promo_timing_level', None)
        charts.pop('promo_timing_level_main', None)

    # Chart 8: Manager vs IC Ratio ('mgmt_ratio') - Donut (Specific layout)
    try:
        fig = None
        if 'role' in comp_df.columns:
            role_data = comp_df.groupby(['job_function', 'role'], observed=False).size().unstack(fill_value=0)
            required_roles = ['Individual Contributor', 'People Manager']
            if ('Product Development' in role_data.index and 'Content and Studio' in role_data.index and
                all(role in role_data.columns for role in required_roles)):

                fig = go.Figure()
                prod_vals = role_data.loc['Product Development']; cont_vals = role_data.loc['Content and Studio']
                prod_ic = prod_vals.get('Individual Contributor', 0); prod_mgr = prod_vals.get('People Manager', 0)
                cont_ic = cont_vals.get('Individual Contributor', 0); cont_mgr = cont_vals.get('People Manager', 0)

                # Add Pie Traces with specific domains and colors
                fig.add_trace(go.Pie(
                    values=[prod_ic, prod_mgr], labels=['IC', 'Manager'], name='Product Dev',
                    domain=dict(x=[0.25, 0.75], y=[0.55, 1.0]), hole=0.4, # Top donut
                    marker_colors=[netflix_grey, netflix_red], # IC=grey, Mgr=red
                    hovertemplate='<b>%{data.name}</b><br>%{label}: %{percent:.1%}<extra></extra>',
                    showlegend=True, textinfo='none'
                ))
                fig.add_trace(go.Pie(
                    values=[cont_ic, cont_mgr], labels=['IC', 'Manager'], name='Content & Studio',
                    domain=dict(x=[0.25, 0.75], y=[0.05, 0.50]), hole=0.4, # Bottom donut
                    marker_colors=[netflix_grey, netflix_red], # IC=grey, Mgr=red
                    hovertemplate='<b>%{data.name}</b><br>%{label}: %{percent:.1%}<extra></extra>',
                    showlegend=True, textinfo='none'
                ))
                # Add annotations for function labels
                annotations = [
                    dict(text="<b>Product <br>Development</b>", x=0.00, y=0.77, showarrow=False, font=dict(size=14), align="center", xref='paper', yref='paper'),
                    dict(text="<b>Content <br>& Studio</b>", x=0.95, y=0.27, showarrow=False, font=dict(size=14), align="center", xref='paper', yref='paper')
                ]
                # Apply specific layout for this chart
                fig.update_layout(
                    annotations=annotations, margin=dict(l=10, r=10, t=30, b=20), height=450, # Adjusted height
                    showlegend=True,
                    legend=dict(orientation="h", yanchor="bottom", y=-0.1, xanchor="center", x=0.5), # Legend below
                    title='Manager vs IC Ratio' # Title set here
                    # Background colors handled by encode_chart
                )
                # Encode this specific layout
                encoded_chart = encode_chart(fig, 'Manager vs IC Ratio') # Pass title again for logging
                if encoded_chart: charts['mgmt_ratio'] = encoded_chart
                else: print("Skipping 'mgmt_ratio': Encoding failed.")
            else: print("Skipping 'mgmt_ratio': Missing required data, functions, or roles.")
        else: print("Skipping 'mgmt_ratio': Missing 'role' column.")
    except Exception as e:
        print(f"Error generating Chart 'mgmt_ratio': {e}"); traceback.print_exc()
        if 'mgmt_ratio' in charts: del charts['mgmt_ratio']

    # Chart 9: Span Distribution ('span_distribution') - Horizontal Bar
    try:
        fig = None
        if 'is_manager' in comp_df.columns and 'span_of_control' in comp_df.columns:
            managers_df = comp_df[comp_df['is_manager']].copy()
            managers_df['span_of_control'] = pd.to_numeric(managers_df['span_of_control'], errors='coerce')
            managers_df.dropna(subset=['span_of_control'], inplace=True)

            if not managers_df.empty:
                bins = [-1, 0, 3, 6, 9, np.inf]; labels = ['0', '1-3', '4-6', '7-9', '10+']
                managers_df['span_bin'] = pd.cut(managers_df['span_of_control'], bins=bins, labels=labels, right=True)
                span_dist_data = managers_df.groupby(['job_function', 'span_bin'], observed=False).size().unstack(fill_value=0)
                # Calculate percentage within each function
                span_dist_pct = span_dist_data.apply(lambda x: x / x.sum() * 100 if x.sum() > 0 else x, axis=1).fillna(0)
                # Melt for table structure
                span_dist_melt = span_dist_pct.reset_index().melt(id_vars='job_function', var_name='span_bin', value_name='percentage')

                fig = px.bar(span_dist_melt, y='span_bin', x='percentage', color='job_function',
                             barmode='group', orientation='h', color_discrete_map=color_map,
                             category_orders={'span_bin': labels[::-1]}, # Reverse order for horizontal
                             labels={'span_bin': 'Span of Control', 'percentage': '% of Managers', 'job_function': 'Function'})
                if fig:
                    fig.update_traces(hovertemplate='<b>%{data.name}</b><br>Span: %{y}<br>% Managers: %{x:.1f}%<extra></extra>')
                    # Override legend position for horizontal bar chart
                    fig.update_layout(legend=dict(orientation='h', yanchor='top', y=-0.30, x=0.5, xanchor='center'))
                    encoded_chart = encode_chart(fig, 'Manager Span Distribution (%)')
                    if encoded_chart: charts['span_distribution'] = encoded_chart
                    else: print("Skipping 'span_distribution': Encoding failed.")
                else: print("Skipping 'span_distribution': Figure object not created.")
            else: print("Skipping 'span_distribution': No managers with valid span data found.")
        else: print("Skipping 'span_distribution': Missing 'span_of_control' or 'is_manager' data.")
    except Exception as e:
        print(f"Error generating Chart 'span_distribution': {e}"); traceback.print_exc()
        if 'span_distribution' in charts: del charts['span_distribution']

    # Chart 10: Leadership Distribution ('leadership_distribution') - Bar
    try:
        fig = None
        if 'job_level_title' in comp_df.columns and 'job_function' in comp_df.columns:
            leadership_levels = ['Director', 'Vice President']
            leadership_df = comp_df[comp_df['job_level_title'].isin(leadership_levels)].copy()

            if not leadership_df.empty:
                total_by_func = comp_df.groupby('job_function', observed=False).size()
                leadership_counts = leadership_df.groupby(['job_function', 'job_level_title'], observed=False).size()
                # Calculate percentage of *total* function employees at these levels
                leadership_pcts = (leadership_counts.div(total_by_func, level='job_function') * 100).reset_index(name='percentage')
                leadership_pcts['percentage'] = leadership_pcts['percentage'].fillna(0)

                fig = px.bar(leadership_pcts, x='job_level_title', y='percentage', color='job_function',
                             barmode='group', color_discrete_map=color_map,
                             category_orders={'job_level_title': leadership_levels},
                             labels={'percentage': '% of Function Employees', 'job_level_title': 'Leadership Level', 'job_function': 'Function'})
                if fig:
                    fig.update_traces(hovertemplate='<b>%{data.name}</b><br>Level: %{x}<br>% of Function: %{y:.1f}%<extra></extra>')
                    encoded_chart = encode_chart(fig, 'Leadership Representation (% of Function)')
                    if encoded_chart: charts['leadership_distribution'] = encoded_chart
                    else: print("Skipping 'leadership_distribution': Encoding failed.")
                else: print("Skipping 'leadership_distribution': Figure object not created.")
            else: print("Skipping 'leadership_distribution': No employees found at Director or VP levels.")
        else: print("Skipping 'leadership_distribution': Missing 'job_level_title' or 'job_function' columns.")
    except Exception as e:
        print(f"Error generating Chart 'leadership_distribution': {e}"); traceback.print_exc()
        if 'leadership_distribution' in charts: del charts['leadership_distribution']

    # Chart 11: Average Tenure by Level ('tenure_level') - Line
    try:
        fig = None
        if 'tenure_years' in comp_df.columns and 'job_level_title' in comp_df.columns:
             # Ensure tenure is numeric
             comp_df['tenure_years_num'] = pd.to_numeric(comp_df['tenure_years'], errors='coerce')
             tenure_level_data = comp_df.dropna(subset=['tenure_years_num']).groupby(
                 ['job_function', 'job_level_title'], observed=False
             )['tenure_years_num'].mean().reset_index()
             # Order the levels
             tenure_level_data['job_level_title'] = pd.Categorical(tenure_level_data['job_level_title'], categories=level_order, ordered=True)
             tenure_level_data = tenure_level_data.sort_values('job_level_title')

             if not tenure_level_data.empty:
                  fig = px.line(tenure_level_data, x='job_level_title', y='tenure_years_num', color='job_function',
                                markers=True, color_discrete_map=color_map,
                                labels={'tenure_years_num': 'Avg Tenure (Yrs)', 'job_level_title': 'Job Level', 'job_function': 'Function'})
                  if fig:
                      fig.update_traces(hovertemplate='<b>%{data.name}</b><br>Level: %{x}<br>Avg Tenure: %{y:.1f} Yrs<extra></extra>')
                      # Apply category order to axis
                      fig.update_layout(xaxis_categoryorder='array', xaxis_categoryarray=level_order)
                      encoded_chart = encode_chart(fig, 'Average Tenure by Level (Trend)')
                      if encoded_chart: charts['tenure_level'] = encoded_chart
                      else: print("Skipping 'tenure_level': Encoding failed.")
                  else: print("Skipping 'tenure_level': Figure object not created.")
             else: print("Skipping 'tenure_level': No data after grouping for line chart.")
        else: print("Skipping 'tenure_level': Missing or insufficient 'tenure_years' or 'job_level_title' data.")
    except Exception as e:
        print(f"Error generating Chart 'tenure_level' (Line): {e}"); traceback.print_exc()
        if 'tenure_level' in charts: del charts['tenure_level']

    # Chart 12: Hiring by Year ('hiring_year') - Bar
    try:
        fig = None
        # Use hire_year_int created earlier if available
        hire_year_col = 'hire_year_int' if 'hire_year_int' in comp_df.columns else 'hire_year'
        if hire_year_col in comp_df.columns and comp_df[hire_year_col].notna().any():
            # Ensure the column is Int64 before grouping
            comp_df[hire_year_col] = pd.to_numeric(comp_df[hire_year_col], errors='coerce').astype('Int64')
            hire_year_data = comp_df.dropna(subset=[hire_year_col]).groupby(
                 [hire_year_col, 'job_function'], observed=False
            ).size().reset_index(name='count')

            if not hire_year_data.empty:
                fig = px.bar(hire_year_data, x=hire_year_col, y='count', color='job_function',
                             barmode='group', color_discrete_map=color_map,
                             labels={'count': '# Hires', hire_year_col: 'Year', 'job_function': 'Function'})
                if fig:
                    fig.update_traces(hovertemplate='<b>%{data.name}</b><br>Year: %{x}<br>Hires: %{y}<extra></extra>')
                    encoded_chart = encode_chart(fig, 'Annual Hiring Volume') # encode_chart handles category axis
                    if encoded_chart: charts['hiring_year'] = encoded_chart
                    else: print("Skipping 'hiring_year': Encoding failed.")
                else: print("Skipping 'hiring_year': Figure object not created.")
            else: print("Skipping 'hiring_year': No valid numeric hire years found after cleaning.")
        else: print(f"Skipping 'hiring_year': Missing '{hire_year_col}' data.")
    except Exception as e:
        print(f"Error generating Chart 'hiring_year': {e}"); traceback.print_exc()
        if 'hiring_year' in charts: del charts['hiring_year']

    # Chart 13: Hiring by Level ('hiring_level') - Bar (% of Hires)
    # *** CORRECTED MELT LOGIC ***
    try:
        fig = None
        hire_year_col = 'hire_year_int' if 'hire_year_int' in comp_df.columns else 'hire_year'
        if hire_year_col in comp_df.columns and comp_df[hire_year_col].notna().any():
            # Define hires based on having a valid hire year
            hires_df = comp_df.dropna(subset=[hire_year_col])
            if not hires_df.empty and 'job_level_title' in hires_df.columns:
                 # Calculate counts per function/level among hires
                 hire_levels = hires_df.groupby(['job_function', 'job_level_title'], observed=False).size().unstack(fill_value=0)
                 # Calculate percentage *within each function's hires*
                 # axis=1 calculates percentages across columns (levels) for each function (row)
                 hire_levels_pct = hire_levels.apply(lambda x: x / x.sum() * 100 if x.sum() > 0 else x, axis=1).reset_index()
                 # Melt: Keep 'job_function' fixed, unpivot the level columns
                 hire_level_melt = hire_levels_pct.melt(id_vars='job_function', var_name='job_level_title', value_name='percentage') # Corrected id_vars
                 # Order levels
                 hire_level_melt['job_level_title'] = pd.Categorical(hire_level_melt['job_level_title'], categories=level_order, ordered=True)
                 hire_level_melt = hire_level_melt.sort_values('job_level_title')

                 fig = px.bar(hire_level_melt, x='job_level_title', y='percentage', color='job_function',
                              barmode='group', color_discrete_map=color_map,
                              labels={'percentage': '% of Function Hires', 'job_level_title': 'Level Hired Into', 'job_function': 'Function'})
                 if fig:
                     fig.update_traces(hovertemplate='<b>%{data.name}</b><br>Level: %{x}<br>% Hires: %{y:.1f}%<extra></extra>')
                     encoded_chart = encode_chart(fig, 'New Hire Distribution by Level (%)')
                     if encoded_chart: charts['hiring_level'] = encoded_chart
                     else: print("Skipping 'hiring_level': Encoding failed.")
                 else: print("Skipping 'hiring_level': Figure object not created.")
            else:
                print("Skipping 'hiring_level': Missing 'job_level_title' data or no valid hires found.")
        else:
            print(f"Skipping 'hiring_level': Missing or invalid '{hire_year_col}' data required to identify hires.")
    except Exception as e:
        print(f"Error generating Chart 'hiring_level': {e}"); traceback.print_exc()
        if 'hiring_level' in charts: del charts['hiring_level']


    # Chart 14: Promotion Timeline Distribution ('promotion_timeline_hist') - Distplot
    try:
        fig = None
        promo_df_hist = pd.DataFrame() # Initialize
        if 'time_to_promotion_num' in comp_df.columns: # Use numeric version created earlier
             promo_df_hist = comp_df[comp_df['time_to_promotion_num'] >= 0].copy()

        if not promo_df_hist.empty:
            hist_data = []
            group_labels = []
            dist_colors = []
            # Ensure functions are sorted for consistent legend order
            sorted_functions = sorted(promo_df_hist['job_function'].unique())
            for function in sorted_functions:
                # Extract TTP data for the function
                func_data = promo_df_hist[promo_df_hist['job_function'] == function]['time_to_promotion_num'].tolist()
                if func_data: # Only add if data exists
                    hist_data.append(func_data)
                    group_labels.append(function)
                    dist_colors.append(color_map.get(function, netflix_other))

            if hist_data: # Check if we collected any data to plot
                fig = ff.create_distplot(
                    hist_data, group_labels, colors=dist_colors,
                    bin_size=0.5, # Adjust bin size as needed
                    show_hist=True, show_curve=True, show_rug=False
                )
                fig.update_layout(
                    xaxis_title='Time to Promotion (Yrs)', yaxis_title='Density',
                    legend_title='Function'
                    # Title set by encode_chart
                )
                encoded_chart = encode_chart(fig, 'Distribution of Time to Promotion (Distplot)')
                if encoded_chart: charts['promotion_timeline_hist'] = encoded_chart
                else: print("Skipping 'promotion_timeline_hist': Encoding failed.")
            else: print("Skipping 'promotion_timeline_hist': No valid data groups found for distplot.")
        else: print("Skipping 'promotion_timeline_hist': Missing or invalid 'time_to_promotion' data.")
    except ImportError: print("Error generating Chart 'promotion_timeline_hist': Plotly Figure Factory (ff) not available or import failed.")
    except Exception as e:
        print(f"Error generating Chart 'promotion_timeline_hist' (Distplot): {e}"); traceback.print_exc()
        if 'promotion_timeline_hist' in charts: del charts['promotion_timeline_hist']


    # Chart 15: Tenure vs. Time to Promotion ('tenure_vs_ttp') - Scatter
    try:
        fig = None
        # Use numeric versions created earlier if available
        ttp_col = 'time_to_promotion_num' if 'time_to_promotion_num' in comp_df.columns else 'time_to_promotion'
        tenure_col = 'tenure_years' # Assuming tenure_years is already numeric from load_data

        if ttp_col in comp_df.columns and tenure_col in comp_df.columns:
             # Ensure both columns are numeric and filter for valid TTP >= 0 and non-null tenure
             comp_df[ttp_col] = pd.to_numeric(comp_df[ttp_col], errors='coerce')
             comp_df[tenure_col] = pd.to_numeric(comp_df[tenure_col], errors='coerce')
             promo_df_scatter = comp_df[(comp_df[ttp_col] >= 0) & comp_df[tenure_col].notna()].copy()
        else: promo_df_scatter = pd.DataFrame()

        if not promo_df_scatter.empty:
            # Add trendline only if sufficient data points exist
            trendline_arg = 'ols' if len(promo_df_scatter) > 10 else None
            fig = px.scatter(promo_df_scatter, x=tenure_col, y=ttp_col, color='job_function',
                             trendline=trendline_arg, trendline_scope="overall", opacity=0.6, color_discrete_map=color_map,
                             labels={tenure_col: 'Tenure at Promo (Yrs)', ttp_col: 'Time Since Last Promo (Yrs)', 'job_function': 'Function'})
            if fig:
                fig.update_traces(hovertemplate=('<b>%{data.name}</b><br>Tenure: %{x:.1f} Yrs<br>TTP: %{y:.1f} Yrs<extra></extra>'))
                encoded_chart = encode_chart(fig, 'Tenure vs. Time to Promotion')
                if encoded_chart: charts['tenure_vs_ttp'] = encoded_chart
                else: print("Skipping 'tenure_vs_ttp': Encoding failed.")
            else: print("Skipping 'tenure_vs_ttp': Figure object not created.")
        else: print("Skipping 'tenure_vs_ttp': Missing or invalid 'time_to_promotion' or 'tenure_years' data.")
    except Exception as e:
        print(f"Error generating Chart 'tenure_vs_ttp': {e}"); traceback.print_exc()
        if 'tenure_vs_ttp' in charts: del charts['tenure_vs_ttp']

    # Chart 16: Average Time to Promotion (Gauge) ('promotion_timing_gauge') - Gauge (Specific layout)
    try:
        fig = None
        ttp_col = 'time_to_promotion_num' if 'time_to_promotion_num' in comp_df.columns else 'time_to_promotion'
        if ttp_col in comp_df.columns:
             comp_df[ttp_col] = pd.to_numeric(comp_df[ttp_col], errors='coerce')
             promo_df_gauge = comp_df[comp_df[ttp_col] >= 0].copy()
        else: promo_df_gauge = pd.DataFrame()

        if not promo_df_gauge.empty:
            prod_avg_ttp = promo_df_gauge.loc[promo_df_gauge['job_function'] == 'Product Development', ttp_col].mean()
            cont_avg_ttp = promo_df_gauge.loc[promo_df_gauge['job_function'] == 'Content and Studio', ttp_col].mean()

            if pd.notna(prod_avg_ttp) and pd.notna(cont_avg_ttp):
                # Determine gauge range dynamically
                max_val = max(prod_avg_ttp, cont_avg_ttp, 0); min_range = 5 # Ensure gauge shows at least 5 years
                max_gauge = max(min_range, np.ceil(max_val * 1.2)) # Add 20% buffer, minimum 5

                fig = go.Figure()
                fig.add_trace(go.Indicator(
                    mode = "gauge+number", value = safe_round(prod_avg_ttp, 1), domain = {'x': [0, 0.48], 'y': [0, 1]},
                    title = {'text': "Product Dev Avg TTP", 'font': {'size': 14}},
                    gauge = {'axis': {'range': [0, max_gauge]}, 'bar': {'color': netflix_red}},
                    number = {'suffix': " yrs", 'font': {'size': 18}}))
                fig.add_trace(go.Indicator(
                    mode = "gauge+number", value = safe_round(cont_avg_ttp, 1), domain = {'x': [0.52, 1.0], 'y': [0, 1]},
                    title = {'text': "Content Avg TTP", 'font': {'size': 14}},
                    gauge = {'axis': {'range': [0, max_gauge]}, 'bar': {'color': netflix_grey}},
                    number = {'suffix': " yrs", 'font': {'size': 18}}))
                # Apply specific layout for gauges
                fig.update_layout(
                    height=250, # Adjust height for gauges
                    # Title handled by encode_chart, margins adjusted by encode_chart for indicators
                )
                encoded_chart = encode_chart(fig, 'Average Time to Promotion')
                if encoded_chart: charts['promotion_timing_gauge'] = encoded_chart
                else: print("Skipping 'promotion_timing_gauge': Encoding failed.")
            else: print("Skipping 'promotion_timing_gauge': Could not calculate valid average TTP for one or both functions.")
        else: print("Skipping 'promotion_timing_gauge': Missing or invalid 'time_to_promotion' data.")
    except Exception as e:
        print(f"Error generating Chart 'promotion_timing_gauge': {e}"); traceback.print_exc()
        if 'promotion_timing_gauge' in charts: del charts['promotion_timing_gauge']

    # Chart 17: Tenure Density ('tenure_density_alt') - Density Line (using histogram approximation)
    try:
        fig = None
        tenure_col = 'tenure_years' # Assuming tenure_years is already numeric from load_data
        if tenure_col in comp_df.columns:
             comp_df[tenure_col] = pd.to_numeric(comp_df[tenure_col], errors='coerce')
             tenure_df_density = comp_df.dropna(subset=[tenure_col])
        else: tenure_df_density = pd.DataFrame()

        if not tenure_df_density.empty:
            fig = go.Figure()
            for function in sorted(tenure_df_density['job_function'].unique()):
                func_data = tenure_df_density[tenure_df_density['job_function'] == function][tenure_col]
                if not func_data.empty and len(func_data) > 1: # Need >1 point for density
                    # Use numpy histogram to approximate density
                    density, bin_edges = np.histogram(func_data, bins=30, density=True) # Adjust bins as needed
                    x_vals = 0.5 * (bin_edges[1:] + bin_edges[:-1]) # Bin midpoints
                    color = color_map.get(function, netflix_other)
                    fig.add_trace(go.Scatter(
                        x=x_vals, y=density, mode='lines', name=function, line=dict(color=color, shape='spline'), # Smoothed line
                        hovertemplate=(f'<b>{function}</b><br>Tenure (approx): %{{x:.1f}} Yrs<br>Density: %{{y:.3f}}<extra></extra>')
                    ))
                elif not func_data.empty: print(f"Warning: Only one data point for {function} in tenure density, skipping.")

            fig.update_layout(xaxis_title='Tenure (Yrs)', yaxis_title='Density', legend_title='Function')
            if fig and fig.data:
                encoded_chart = encode_chart(fig, 'Tenure Density by Function')
                if encoded_chart: charts['tenure_density_alt'] = encoded_chart
                else: print("Skipping 'tenure_density_alt': Encoding failed.")
            else: print("Skipping 'tenure_density_alt': Figure object created but has no data or insufficient data.")
        else: print("Skipping 'tenure_density_alt': Missing or insufficient 'tenure_years' data.")
    except Exception as e:
        print(f"Error generating Chart 'tenure_density_alt': {e}"); traceback.print_exc()
        if 'tenure_density_alt' in charts: del charts['tenure_density_alt']

    # Chart 18: Average Span by Manager Level ('span_level_chart') - Line
    try:
        fig = None
        if 'is_manager' in comp_df.columns and 'span_of_control' in comp_df.columns and 'job_level_title' in comp_df.columns:
            # Ensure span is numeric, filter for managers with span > 0
            managers_df = comp_df[comp_df['is_manager']].copy()
            managers_df['span_of_control_num'] = pd.to_numeric(managers_df['span_of_control'], errors='coerce')
            managers_df = managers_df[managers_df['span_of_control_num'] > 0]

            if not managers_df.empty:
                span_level_data = managers_df.groupby(['job_function', 'job_level_title'], observed=False)['span_of_control_num'].mean().reset_index()
                # Order by specific manager levels
                span_level_data['job_level_title'] = pd.Categorical(span_level_data['job_level_title'], categories=manager_level_order, ordered=True)
                span_level_data = span_level_data.dropna(subset=['job_level_title']).sort_values('job_level_title')

                if not span_level_data.empty:
                     fig = px.line(span_level_data, x='job_level_title', y='span_of_control_num', color='job_function',
                                   markers=True, color_discrete_map=color_map,
                                   labels={'span_of_control_num': 'Avg Span (Mgrs w/ Span>0)', 'job_level_title': 'Manager Level', 'job_function': 'Function'})
                     if fig:
                         fig.update_traces(hovertemplate='<b>%{data.name}</b><br>Level: %{x}<br>Avg Span: %{y:.1f}<extra></extra>')
                         fig.update_layout(xaxis_categoryorder='array', xaxis_categoryarray=manager_level_order) # Ensure correct order on axis
                         encoded_chart = encode_chart(fig, 'Average Span by Manager Level (Trend)')
                         if encoded_chart: charts['span_level_chart'] = encoded_chart
                         else: print("Skipping 'span_level_chart': Encoding failed.")
                     else: print("Skipping 'span_level_chart': Figure object not created.")
                else: print("Skipping 'span_level_chart': No valid data after filtering for manager levels.")
            else: print("Skipping 'span_level_chart': No managers with span > 0 found.")
        else: print("Skipping 'span_level_chart': Missing required columns.")
    except Exception as e:
        print(f"Error generating Chart 'span_level_chart' (Line): {e}"); traceback.print_exc()
        if 'span_level_chart' in charts: del charts['span_level_chart']

    # Chart 19: Approximate Promotion Rate Trend ('promotion_rate_chart') - Line
    try:
        fig = None
        # Requires promotion status, promotion year, and hire year
        promo_year_col = 'promotion_year_int' if 'promotion_year_int' in comp_df.columns else 'promotion_year'
        hire_year_col = 'hire_year_int' if 'hire_year_int' in comp_df.columns else 'hire_year'

        if ('is_promoted' in comp_df.columns and promo_year_col in comp_df.columns and
            hire_year_col in comp_df.columns and comp_df[promo_year_col].notna().any() and
            comp_df[hire_year_col].notna().any()):

            # --- Calculate Promotions per Year/Function ---
            promo_df = comp_df[comp_df['is_promoted']].copy()
            # Ensure year is Int64 before grouping
            promo_df[promo_year_col] = pd.to_numeric(promo_df[promo_year_col], errors='coerce').astype('Int64')
            promo_counts = promo_df.dropna(subset=[promo_year_col]).groupby(
                [promo_year_col, 'job_function'], observed=False
            ).size()

            # --- Calculate Cumulative Headcount per Year/Function ---
            # Ensure hire_year is Int64 before grouping
            temp_comp_df = comp_df.copy()
            temp_comp_df[hire_year_col] = pd.to_numeric(temp_comp_df[hire_year_col], errors='coerce').astype('Int64')
            # Group by hire year/function, sum, then calculate cumulative sum over years
            total_counts = temp_comp_df.dropna(subset=[hire_year_col]).groupby(
                 [hire_year_col, 'job_function'], observed=False
            ).size().unstack(fill_value=0).cumsum().stack()

            # --- Align and Calculate Rate ---
            if not promo_counts.empty and not total_counts.empty:
                # Determine common index (all years/functions present in either series)
                all_years = sorted(list(set(promo_counts.index.get_level_values(0).dropna()) | set(total_counts.index.get_level_values(0).dropna())))
                all_funcs = sorted(list(set(promo_counts.index.get_level_values(1)) | set(total_counts.index.get_level_values(1))))
                multi_index = pd.MultiIndex.from_product([all_years, all_funcs], names=['year', 'job_function'])

                # Reindex both series to the common index, filling missing values with 0
                promo_counts_aligned = promo_counts.reindex(multi_index, fill_value=0)
                total_counts_aligned = total_counts.reindex(multi_index, fill_value=0)

                # Calculate rate, handle division by zero (replace Inf with 0 or NaN)
                rate_data = (promo_counts_aligned / total_counts_aligned * 100).replace([np.inf, -np.inf], 0).fillna(0).reset_index(name='rate')

                # Filter out years where total count was zero (leading to 0 rate) if desired, or keep them
                # rate_data = rate_data[total_counts_aligned.reindex(multi_index, fill_value=0).reset_index(name='total')['total'] > 0]

                if not rate_data.empty:
                    fig = px.line(rate_data, x='year', y='rate', color='job_function',
                                  markers=True, color_discrete_map=color_map,
                                  labels={'rate': 'Approx. Promotion Rate (%)', 'year': 'Year', 'job_function': 'Function'})
                    if fig:
                        fig.update_traces(hovertemplate='<b>%{data.name}</b><br>Year: %{x}<br>Rate: %{y:.1f}%<extra></extra>')
                        fig.update_layout(yaxis_ticksuffix='%')
                        encoded_chart = encode_chart(fig, 'Approximate Annual Promotion Rate (%)') # encode_chart handles category axis
                        if encoded_chart: charts['promotion_rate_chart'] = encoded_chart
                        else: print("Skipping 'promotion_rate_chart': Encoding failed.")
                    else: print("Skipping 'promotion_rate_chart': Figure object not created.")
                else: print("Skipping 'promotion_rate_chart': No valid rate data after calculation/filtering.")
            else: print("Skipping 'promotion_rate_chart': Insufficient promotion or total count data.")
        else: print(f"Skipping 'promotion_rate_chart': Missing required promotion_year, hire_year, or is_promoted data.")
    except Exception as e:
        print(f"Error generating Chart 'promotion_rate_chart': {e}"); traceback.print_exc()
        if 'promotion_rate_chart' in charts: del charts['promotion_rate_chart']

    # Chart 20: Promotions by Quarter ('promotion_quarter_chart') - Faceted Bar
    try:
        fig = None
        chart_title = ""
        # Use datetime version of promotion date
        promo_date_col = 'promotion_date_dt'
        if promo_date_col in comp_df.columns and 'is_promoted' in comp_df.columns:
            # Filter for promoted employees with valid datetime objects
            promo_q_df = comp_df[comp_df['is_promoted'] & comp_df[promo_date_col].notna()].copy()

            if not promo_q_df.empty:
                # Extract year and quarter
                promo_q_df['promotion_year_str'] = promo_q_df[promo_date_col].dt.year.astype(str)
                promo_q_df['promotion_quarter_str'] = 'Q' + promo_q_df[promo_date_col].dt.quarter.astype(str)
                # Group by quarter, year, function
                promo_q_data = promo_q_df.groupby(
                    ['promotion_quarter_str', 'promotion_year_str', 'job_function'], observed=False
                ).size().reset_index(name='count')
                # Define order for quarters and years
                unique_years = sorted(promo_q_data['promotion_year_str'].unique())
                quarter_order = ['Q1', 'Q2', 'Q3', 'Q4']
                # Apply categorical ordering
                promo_q_data['promotion_quarter_str'] = pd.Categorical(promo_q_data['promotion_quarter_str'], categories=quarter_order, ordered=True)
                promo_q_data = promo_q_data.sort_values(['promotion_year_str', 'promotion_quarter_str'])

                if len(unique_years) == 1: # Single year -> Simple bar chart
                    year_label = unique_years[0]
                    fig = px.bar(promo_q_data, x='promotion_quarter_str', y='count', color='job_function',
                                 barmode='group', color_discrete_map=color_map,
                                 labels={'promotion_quarter_str': 'Quarter', 'count': '# Promotions', 'job_function': 'Function'})
                    chart_title = f'Promotions by Quarter ({year_label})'
                elif len(unique_years) > 1: # Multiple years -> Faceted bar chart
                    wrap_cols = 2 if len(unique_years) <= 4 else 3 # Adjust wrap based on number of years
                    fig = px.bar(promo_q_data, x='promotion_quarter_str', y='count', color='job_function',
                                 barmode='group', facet_col='promotion_year_str', facet_col_wrap=wrap_cols,
                                 category_orders={'promotion_year_str': unique_years, 'promotion_quarter_str': quarter_order},
                                 color_discrete_map=color_map,
                                 labels={'promotion_quarter_str': 'Quarter', 'count': '# Promotions', 'job_function': 'Function'})
                    chart_title = 'Promotions by Quarter (Faceted by Year)'
                    # Remove individual axis titles for facets for cleaner look
                    fig.for_each_xaxis(lambda axis: axis.update(title_text=''))
                    fig.for_each_yaxis(lambda axis: axis.update(title_text=''))
                else: print("Skipping 'promotion_quarter_chart': No unique years found.")

                if fig: # Check if fig was successfully created
                    fig.update_traces(hovertemplate='<b>%{data.name}</b><br>Quarter: %{x}<br>Promotions: %{y}<extra></extra>')
                    encoded_chart = encode_chart(fig, chart_title) # encode_chart handles facet label cleaning
                    if encoded_chart: charts['promotion_quarter_chart'] = encoded_chart
                    else: print(f"Skipping '{chart_title}': Encoding failed.")
            else: print("Skipping 'promotion_quarter_chart': No valid promotion dates or no promotions.")
        else: print(f"Skipping 'promotion_quarter_chart': Missing '{promo_date_col}' or 'is_promoted' column.")
    except Exception as e:
        print(f"Error generating Chart 'promotion_quarter_chart': {e}"); traceback.print_exc()
        if 'promotion_quarter_chart' in charts: del charts['promotion_quarter_chart']

    print(f"Generated {len(charts)} chart specifications.")
    return charts


# =============================================================================
# # 7. Summary Statistics Generation
# =============================================================================
# Description: This section defines the `generate_summary_stats` function.
# It calculates key aggregate metrics (total employees, avg tenure, avg span,
# avg TTP) overall and by function. It also calculates Year-over-Year (YoY)
# changes comparing 2023 vs 2022 for hiring counts, promotion counts, average TTP
# for promotions in those years, and the difference in *current* average tenure
# between the 2023 and 2022 hire cohorts. These calculated YoY values are
# assigned to the specific keys expected by the HTML template's KPI cards.
# YoY changes requiring full historical snapshots (like YoY change in total active
# headcount or avg span of the *current* workforce) are set to None.


def generate_summary_stats(df):
    """
    Calculates summary statistics for the dashboard's key metric displays.

    Computes overall and function-specific averages for tenure, time-to-promotion,
    and span of control. Calculates YoY changes (2023 vs 2022) for hiring counts,
    promotion counts, average TTP (for promotions *in* those years), and the
    difference in current average tenure between 2023 and 2022 hire cohorts,
    assigning them to the specific keys expected by the template's KPI cards.
    Sets YoY changes requiring historical snapshots to None.

    Args:
        df (pd.DataFrame): The processed DataFrame from `load_data`.

    Returns:
        dict: A dictionary containing calculated summary statistics, including
              YoY metrics mapped to specific template keys. Returns an empty
              dictionary if the input DataFrame is invalid.
    """
    summary = {}
    if df is None or df.empty:
        print("Warning: Input DataFrame is empty for summary stats generation.")
        return summary

    # Filter for primary functions, falling back if necessary
    comp_df = df[df['job_function'].isin(['Product Development', 'Content and Studio'])].copy()
    if comp_df.empty:
        print("Warning: No data for primary functions found for summary stats. Using all data.")
        comp_df = df.copy() # Use original if filter is empty

    if comp_df.empty:
        print("Warning: DataFrame is empty even after fallback for summary stats.")
        return summary

    # --- Overall Metrics (Based on Current Snapshot/Data) ---
    summary['total_employees'] = len(comp_df)
    # Calculate overall average tenure of *current* workforce
    if 'tenure_years' in comp_df.columns:
        summary['avg_tenure'] = comp_df['tenure_years'].mean()

    # Calculate overall average time-to-promotion (for promotions that occurred)
    comp_df['time_to_promotion_num'] = pd.to_numeric(comp_df['time_to_promotion'], errors='coerce')
    promo_df_overall = comp_df[comp_df['time_to_promotion_num'] >= 0].copy()
    summary['avg_time_to_promotion'] = promo_df_overall['time_to_promotion_num'].mean() if not promo_df_overall.empty else None

    # Calculate overall average span of control (for *current* managers with reports)
    managers_df_overall = comp_df[comp_df['is_manager']].copy()
    managers_df_overall['span_of_control_num'] = pd.to_numeric(managers_df_overall['span_of_control'], errors='coerce')
    summary['avg_span'] = managers_df_overall[managers_df_overall['span_of_control_num'] > 0]['span_of_control_num'].mean() if not managers_df_overall.empty else None

    # --- Function-Specific Metrics (Based on Current Snapshot/Data) ---
    product_df = comp_df[comp_df['job_function'] == 'Product Development']
    content_df = comp_df[comp_df['job_function'] == 'Content and Studio']

    # Average Tenure by Function
    if 'tenure_years' in product_df.columns: summary['product_avg_tenure'] = product_df['tenure_years'].mean()
    if 'tenure_years' in content_df.columns: summary['content_avg_tenure'] = content_df['tenure_years'].mean()

    # Average Span by Function (for *current* managers with reports)
    prod_managers = product_df[product_df['is_manager']]
    cont_managers = content_df[content_df['is_manager']]
    if not prod_managers.empty and 'span_of_control_num' in prod_managers.columns:
        summary['product_avg_span'] = prod_managers[prod_managers['span_of_control_num'] > 0]['span_of_control_num'].mean()
    if not cont_managers.empty and 'span_of_control_num' in cont_managers.columns:
        summary['content_avg_span'] = cont_managers[cont_managers['span_of_control_num'] > 0]['span_of_control_num'].mean()

    # Average Time-to-Promotion by Function (for promotions that occurred)
    prod_promo = product_df[product_df['time_to_promotion_num'] >= 0]
    cont_promo = content_df[content_df['time_to_promotion_num'] >= 0]
    summary['product_avg_time_to_promotion'] = prod_promo['time_to_promotion_num'].mean() if not prod_promo.empty else None
    summary['content_avg_time_to_promotion'] = cont_promo['time_to_promotion_num'].mean() if not cont_promo.empty else None

    # --- Calculated Year-over-Year Metrics (2023 vs 2022) ---
    last_year = 2023
    prior_year = 2022

    # Initialize calculated YoY values
    yoy_hires_change_pct_calc = None
    yoy_promotions_change_pct_calc = None
    yoy_avg_ttp_change_calc = None # Absolute change in years for TTP
    yoy_tenure_diff_calc = None # Absolute change in years for Tenure comparison

    # Ensure necessary columns are numeric Int64 for year comparisons
    if 'hire_year' in comp_df.columns:
        comp_df['hire_year_int'] = pd.to_numeric(comp_df['hire_year'], errors='coerce').astype('Int64')
    if 'promotion_year' in comp_df.columns:
        comp_df['promotion_year_int'] = pd.to_numeric(comp_df['promotion_year'], errors='coerce').astype('Int64')

    # YoY Hires Calculation
    if 'hire_year_int' in comp_df.columns:
        hires_last_year = comp_df[comp_df['hire_year_int'] == last_year].shape[0]
        hires_prior_year = comp_df[comp_df['hire_year_int'] == prior_year].shape[0]
        if hires_prior_year > 0:
            yoy_hires_change_pct_calc = ((hires_last_year - hires_prior_year) / hires_prior_year) * 100
        elif hires_last_year > 0:
            yoy_hires_change_pct_calc = np.inf # Indicate increase from zero
        elif hires_prior_year == 0 and hires_last_year == 0:
             yoy_hires_change_pct_calc = 0.0 # No change from zero

    # YoY Promotions Calculation
    promo_df = pd.DataFrame() # Initialize empty df
    if 'promotion_year_int' in comp_df.columns and 'is_promoted' in comp_df.columns:
        promo_df = comp_df[comp_df['is_promoted'] & comp_df['promotion_year_int'].notna()].copy()
        promos_last_year = promo_df[promo_df['promotion_year_int'] == last_year].shape[0]
        promos_prior_year = promo_df[promo_df['promotion_year_int'] == prior_year].shape[0]
        if promos_prior_year > 0:
            yoy_promotions_change_pct_calc = ((promos_last_year - promos_prior_year) / promos_prior_year) * 100
        elif promos_last_year > 0:
            yoy_promotions_change_pct_calc = np.inf # Indicate increase from zero
        elif promos_prior_year == 0 and promos_last_year == 0:
             yoy_promotions_change_pct_calc = 0.0 # No change from zero

    # YoY Average TTP Calculation (for promotions *in* 2023 vs *in* 2022)
    if not promo_df.empty and 'time_to_promotion_num' in promo_df.columns:
         ttp_last_year_df = promo_df[(promo_df['promotion_year_int'] == last_year) & (promo_df['time_to_promotion_num'] >= 0)]
         ttp_prior_year_df = promo_df[(promo_df['promotion_year_int'] == prior_year) & (promo_df['time_to_promotion_num'] >= 0)]
         avg_ttp_last_year = ttp_last_year_df['time_to_promotion_num'].mean() if not ttp_last_year_df.empty else np.nan
         avg_ttp_prior_year = ttp_prior_year_df['time_to_promotion_num'].mean() if not ttp_prior_year_df.empty else np.nan
         if pd.notna(avg_ttp_last_year) and pd.notna(avg_ttp_prior_year):
             yoy_avg_ttp_change_calc = avg_ttp_last_year - avg_ttp_prior_year

    # YoY Tenure Comparison (Comparing *current* avg tenure of 2023 hires vs 2022 hires)
    if 'hire_year_int' in comp_df.columns and 'tenure_years' in comp_df.columns:
        tenure_last_year_hires = comp_df.loc[comp_df['hire_year_int'] == last_year, 'tenure_years'].mean()
        tenure_prior_year_hires = comp_df.loc[comp_df['hire_year_int'] == prior_year, 'tenure_years'].mean()
        if pd.notna(tenure_last_year_hires) and pd.notna(tenure_prior_year_hires):
            # Calculate the difference in current average tenure between the two cohorts
            yoy_tenure_diff_calc = tenure_last_year_hires - tenure_prior_year_hires

    # --- Assign Calculated or Placeholder Values to Template Keys ---

    # KPI Card 1: Total Employees -> Use YoY Hire % Change
    summary['yoy_employee_change_pct'] = yoy_hires_change_pct_calc

    # KPI Card 2: Avg. Tenure -> Use Difference in Avg Current Tenure (2023 hires vs 2022 hires)
    summary['yoy_tenure_change'] = yoy_tenure_diff_calc # Absolute difference in years

    # KPI Card 3: Avg. Span -> Use YoY Promotion % Change (Formatted String)
    yoy_promo_desc = None
    if yoy_promotions_change_pct_calc is not None:
        if yoy_promotions_change_pct_calc == np.inf:
            yoy_promo_desc = "Increase from Zero"
        elif yoy_promotions_change_pct_calc == 0.0:
            yoy_promo_desc = "No Change"
        else:
            yoy_promo_desc = f"{'+' if yoy_promotions_change_pct_calc > 0 else ''}{safe_round(yoy_promotions_change_pct_calc, 1)}%"
    summary['yoy_span_change_desc'] = yoy_promo_desc # Assign formatted string or None

    # KPI Card 4: Avg. TTP -> Use calculated YoY Avg TTP Change (for promotions *in* those years)
    summary['yoy_avg_ttp_change'] = yoy_avg_ttp_change_calc # Absolute change in years

    # --- Clean Final Dictionary for JSON ---
    summary_cleaned = {}
    for k, v in summary.items():
        if pd.isna(v):
            summary_cleaned[k] = None
        elif v == np.inf:
            summary_cleaned[k] = "Increase from Zero"
        elif v == -np.inf:
             summary_cleaned[k] = None # Should not happen
        else:
            summary_cleaned[k] = v

    return summary_cleaned

# =============================================================================
# # 8. Table Data Generation
# =============================================================================
# Description: This section defines the `generate_table_data` function, which
# prepares and formats data specifically for display in the various tables
# within the HTML dashboard (e.g., composition breakdown, tenure metrics,
# span distribution, promotion counts, hiring counts, TTP by level). It
# structures the data as lists of dictionaries, suitable for easy iteration
# within the Jinja2 template.

def generate_table_data(df):
    """
    Generates data structured for rendering tables in the HTML dashboard.

    Prepares data subsets and formats them into lists of dictionaries
    for tables related to composition, tenure, span, promotions, hiring,
    and time-to-promotion.

    Args:
        df (pd.DataFrame): The processed DataFrame from `load_data`.

    Returns:
        dict: A dictionary where keys match the expected table data variables
              in the Jinja2 template (e.g., 'compositionTableData') and values
              are lists of dictionaries representing table rows. Returns an
              empty dictionary if the input DataFrame is invalid.
    """
    table_data = {}
    if df is None or df.empty:
        print("Warning: No data available for table generation.")
        return table_data

    # Filter for primary functions, falling back if necessary
    comp_df = df[df['job_function'].isin(['Product Development', 'Content and Studio'])].copy()
    if comp_df.empty:
        print("Warning: No data for primary functions found for tables. Using all data.")
        comp_df = df.copy() # Use original if filter is empty

    if comp_df.empty:
        print("Warning: DataFrame is empty even after fallback for table data.")
        return table_data

    # --- Composition Table Data (Function x Level Counts) ---
    try:
        # Group by function and level, get counts
        comp_counts = comp_df.groupby(['job_function', 'job_level_title'], observed=False).size().reset_index(name='count')
        # Rename columns for template consistency
        table_data['compositionTableData'] = comp_counts.rename(
            columns={'job_function': 'function', 'job_level_title': 'level'}
        ).to_dict(orient='records')
    except Exception as e:
        print(f"Error generating composition table data: {e}")
        table_data['compositionTableData'] = []

    # --- Tenure Table Data (Function x Metrics) ---
    try:
        if 'tenure_years' in comp_df.columns and comp_df['tenure_years'].notna().any():
            # Calculate mean and median tenure by function
            tenure_summary = comp_df.groupby('job_function', observed=False)['tenure_years'].agg(['mean', 'median']).reset_index()
            tenure_summary.rename(columns={'job_function': 'function', 'mean': 'Average Tenure (Yrs)', 'median': 'Median Tenure (Yrs)'}, inplace=True)

            # Calculate % < 2 years tenure by function
            short_tenure = comp_df[comp_df['tenure_years'] < 2].groupby('job_function', observed=False).size()
            total_tenure = comp_df.groupby('job_function', observed=False).size()
            pct_short_tenure = (short_tenure / total_tenure * 100).fillna(0).reset_index(name='% < 2 Yrs Tenure')
            pct_short_tenure.rename(columns={'job_function': 'function'}, inplace=True)

            # Combine metrics into a single long-format structure for the table
            tenure_avg_med = tenure_summary.melt(id_vars='function', var_name='metric', value_name='value')
            tenure_pct = pct_short_tenure.melt(id_vars='function', var_name='metric', value_name='value')
            tenure_combined = pd.concat([tenure_avg_med, tenure_pct], ignore_index=True)

            # Apply safe rounding (1 decimal for years, 0 for percentage)
            global safe_round
            tenure_combined['value'] = tenure_combined.apply(
                lambda row: safe_round(row['value'], decimals=1 if '(Yrs)' in row['metric'] else 0), axis=1
            )
            table_data['tenureTableData'] = tenure_combined.to_dict(orient='records')
        else:
             table_data['tenureTableData'] = []
    except Exception as e:
        print(f"Error generating tenure table data: {e}")
        table_data['tenureTableData'] = []

    # --- Span Table Data (Function x Span Bin %) ---
    try:
        if 'is_manager' in comp_df.columns and 'span_of_control' in comp_df.columns:
            managers_df = comp_df[comp_df['is_manager']].copy()
            managers_df['span_of_control'] = pd.to_numeric(managers_df['span_of_control'], errors='coerce')
            managers_df.dropna(subset=['span_of_control'], inplace=True)

            if not managers_df.empty:
                # Bin managers by span of control
                bins = [-1, 0, 3, 6, 9, np.inf]; labels = ['0', '1-3', '4-6', '7-9', '10+']
                managers_df['span_bin'] = pd.cut(managers_df['span_of_control'], bins=bins, labels=labels, right=True)
                # Calculate counts per function/bin
                span_dist_data = managers_df.groupby(['job_function', 'span_bin'], observed=False).size().unstack(fill_value=0)
                # Calculate percentage distribution within each function
                span_dist_pct = span_dist_data.apply(lambda x: x / x.sum() * 100 if x.sum() > 0 else x, axis=1).fillna(0)
                # Melt for table structure
                span_dist_melt = span_dist_pct.reset_index().melt(id_vars='job_function', var_name='span_bin', value_name='percentage')
                # Round percentage and rename columns
                span_dist_melt['percentage'] = span_dist_melt['percentage'].apply(safe_round, decimals=1)
                table_data['spanTableData'] = span_dist_melt.rename(columns={'job_function': 'function'}).to_dict(orient='records')
            else:
                 table_data['spanTableData'] = []
        else:
             table_data['spanTableData'] = []
    except Exception as e:
        print(f"Error generating span table data: {e}")
        table_data['spanTableData'] = []

    # --- Promotions Table Data (Year x Function Counts) ---
    try:
        if 'is_promoted' in comp_df.columns and 'promotion_year' in comp_df.columns:
            promo_df = comp_df[comp_df['is_promoted']].copy()
            # Ensure year is numeric Int64
            promo_df['promotion_year'] = pd.to_numeric(promo_df['promotion_year'], errors='coerce').astype('Int64')
            promo_df.dropna(subset=['promotion_year'], inplace=True)

            if not promo_df.empty:
                # Group by year and function, count promotions
                promo_counts = promo_df.groupby(['promotion_year', 'job_function'], observed=False).size().reset_index(name='count')
                # Rename columns for template
                table_data['promotionsTableData'] = promo_counts.rename(
                    columns={'promotion_year': 'year', 'job_function': 'function'}
                ).to_dict(orient='records')
            else:
                 table_data['promotionsTableData'] = []
        else:
             table_data['promotionsTableData'] = []
    except Exception as e:
        print(f"Error generating promotions table data: {e}")
        table_data['promotionsTableData'] = []

    # --- Hiring Table Data (Quarter x Function Counts) ---
    try:
        # Use the hire_year_quarter_str column if previously created, otherwise create it
        if 'hire_year_quarter_str' not in comp_df.columns:
             if 'hire_year' in comp_df.columns and 'hire_quarter' in comp_df.columns:
                 comp_df['hire_year_num'] = pd.to_numeric(comp_df['hire_year'], errors='coerce').astype('Int64')
                 comp_df['hire_year_quarter_str'] = np.where(
                      comp_df['hire_year_num'].notna() & comp_df['hire_quarter'].notna(),
                      comp_df['hire_year_num'].astype(str) + 'Q' + comp_df['hire_quarter'].astype(str).str.replace(r'\.0$', '', regex=True),
                      pd.NA
                 )
             else:
                 comp_df['hire_year_quarter_str'] = pd.NA # Ensure column exists even if data is missing

        if 'hire_year_quarter_str' in comp_df.columns and comp_df['hire_year_quarter_str'].notna().any():
            # Group by year-quarter string and function
            hire_counts = comp_df.dropna(subset=['hire_year_quarter_str']).groupby(
                ['hire_year_quarter_str', 'job_function'], observed=False
            ).size().reset_index(name='count')
            # Sort chronologically
            hire_counts = hire_counts.sort_values('hire_year_quarter_str')
            # Rename columns
            table_data['hiringTableData'] = hire_counts.rename(
                columns={'hire_year_quarter_str': 'quarter', 'job_function': 'function'}
            ).to_dict(orient='records')
        else:
            table_data['hiringTableData'] = []
    except Exception as e:
        print(f"Error generating hiring table data: {e}")
        table_data['hiringTableData'] = []

    # --- Time to Promotion (TTP) Table Data (Function x Level Avg TTP) ---
    try:
        # Use numeric TTP column created earlier if available
        ttp_col = 'time_to_promotion_num' if 'time_to_promotion_num' in comp_df.columns else 'time_to_promotion'
        if ttp_col in comp_df.columns and 'job_level_title' in comp_df.columns:
            # Ensure TTP is numeric and filter for valid TTP >= 0
            comp_df[ttp_col] = pd.to_numeric(comp_df[ttp_col], errors='coerce')
            promo_df_ttp = comp_df[comp_df[ttp_col] >= 0].copy()

            if not promo_df_ttp.empty:
                # Calculate average TTP by function and level promoted into
                ttp_avg = promo_df_ttp.groupby(['job_function', 'job_level_title'], observed=False)[ttp_col].mean().reset_index(name='avg_ttp')
                # Round average TTP
                ttp_avg['avg_ttp'] = ttp_avg['avg_ttp'].apply(safe_round, decimals=1)
                # Filter for relevant manager/director levels for the table display
                ttp_avg_filtered = ttp_avg[ttp_avg['job_level_title'].isin(['Manager', 'Director', 'Vice President'])]
                # Rename columns
                table_data['ttpTableData'] = ttp_avg_filtered.rename(
                    columns={'job_function': 'function', 'job_level_title': 'level'}
                ).to_dict(orient='records')
            else:
                table_data['ttpTableData'] = []
        else:
            table_data['ttpTableData'] = []
    except Exception as e:
        print(f"Error generating TTP table data: {e}")
        table_data['ttpTableData'] = []

    print(f"Generated data for {len(table_data)} tables.")
    return table_data

# =============================================================================
# # 9. HTML Rendering
# =============================================================================
# Description: This section defines the `render_html_template` function, which
# uses the Jinja2 templating engine to populate an HTML template file with the
# generated data (summaries, chart specifications, narratives, table data).
# It writes the final, populated HTML content to the specified output file.


def render_html_template(context, template_filename, output_filename):
    """
    Renders the Jinja2 HTML template with the provided context data.

    Loads the specified template file, injects the context dictionary
    (containing JSON strings of summaries, charts, narratives, etc.),
    and writes the rendered HTML to the output file.

    Args:
        context (dict): Dictionary containing data to pass to the template.
                        Keys should match variables used in the template.
        template_filename (str): The name of the Jinja2 template file.
        output_filename (str): The name of the HTML file to generate.

    Returns:
        bool: True if rendering and writing were successful, False otherwise.
    """
    try:
        # Set up Jinja2 environment to load templates from the current directory
        template_loader = jinja2.FileSystemLoader(searchpath="./")
        # Enable autoescaping for security
        template_env = jinja2.Environment(loader=template_loader, autoescape=jinja2.select_autoescape(['html', 'xml']))

        # Load the specified template
        template = template_env.get_template(template_filename)

        # Render the template with the provided context
        html_output = template.render(context)

        # Write the rendered HTML to the output file
        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write(html_output)

        print(f"Successfully rendered template to {output_filename}")
        return True
    except jinja2.TemplateNotFound:
        print(f"Error: Template file '{template_filename}' not found in the current directory.")
        return False
    except Exception as e:
        print(f"Error rendering HTML template: {e}")
        traceback.print_exc()
        return False

# =============================================================================
# # 10. Main Execution Block
# =============================================================================
# Description: This section defines the `main` function, which serves as the
# entry point and orchestrator for the entire script. It calls the functions
# for data loading, chart generation, narrative creation, summary statistics,
# table data preparation, JSON validation, and finally, HTML rendering.
# It includes error handling and ensures the script runs only when executed directly.


# Helper function to handle np.inf during JSON serialization if needed,
# as fallback for the cleaning step in generate_summary_stats.
def handle_inf_default(obj):
    if obj == np.inf:
        return "Increase from Zero" # Consistent string representation
    elif obj == -np.inf:
         return "Decrease from Zero" # Or handle appropriately
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")


def main():
    """
    Main function to execute the dashboard generation workflow.

    Calls functions sequentially to:
    1. Load and process data.
    2. Generate summary statistics (including calculated YoY metrics).
    3. Create chart specifications.
    4. Generate detailed narratives.
    5. Prepare data for tables.
    6. Validate all generated JSON components.
    7. Render the final HTML dashboard using a Jinja2 template.
    Includes error handling for data loading and rendering steps.
    """
    global report_date, snapshot_file, history_file, template_file, output_html

    print("--- Starting Netflix Org Dashboard Generation ---")
    # Access global report_date (defined in Part 1)
    if 'report_date' not in globals():
         report_date = dt.datetime.now() # Define if somehow missing
    print(f"Report date: {report_date.strftime('%Y-%m-%d')}")

    # --- Step 1: Load and Process Data ---
    df, bad_dates_json, promo_issue_json = load_data()

    # Proceed only if data loading was successful and returned a DataFrame
    if df is not None and not df.empty:
        try:
            print("Generating dashboard components...")

            # --- Step 2-5: Generate Dashboard Components ---
            summary_stats = generate_summary_stats(df) # Includes calculated YoY
            charts = create_charts(df) # Returns dict of chart specs
            narratives = generate_detailed_narratives(df, bad_dates_json, promo_issue_json)
            table_data = generate_table_data(df)

            # --- Step 6: Convert Components to JSON Strings ---
            # Use default=str for robust serialization of common types (e.g., datetime if not handled)
            # The cleaning in generate_summary_stats handles np.inf, but default=str is good practice.
            summary_json_string = json.dumps(summary_stats, default=str)
            valid_charts = {k: v for k, v in charts.items() if v is not None}
            chart_json_string = json.dumps(valid_charts) # Plotly JSON handled by encode_chart
            narratives_cleaned = {k: v for k, v in narratives.items() if v is not None}
            narrative_json_string = json.dumps(narratives_cleaned, default=str)
            table_data_json_string = json.dumps(table_data, default=str)

            # --- Step 7: Validate Generated JSON ---
            print("\n--- Validating Generated JSON ---")
            is_summary_valid = validate_json(summary_json_string, "summary_json")
            is_chart_data_valid = validate_json(chart_json_string, "initial_charts_json")
            is_narrative_valid = validate_json(narrative_json_string, "narratives_json")
            is_bad_dates_valid = validate_json(bad_dates_json, "bad_dates_json")
            is_promo_issue_valid = validate_json(promo_issue_json, "promo_issue_json")
            is_table_data_valid = validate_json(table_data_json_string, "table_data_json")
            print("--- JSON Validation Complete ---\n")

            # --- Step 8: Render HTML (Conditional on JSON Validation) ---
            all_json_valid = (
                is_summary_valid and is_chart_data_valid and is_narrative_valid and
                is_bad_dates_valid and is_promo_issue_valid and is_table_data_valid
            )

            if all_json_valid:
                # Prepare context dictionary for Jinja2 template
                # Globals already declared at function start
                context = {
                    "report_date_str": report_date.strftime('%Y-%m-%d %H:%M'),
                    "snapshot_file": snapshot_file,
                    "history_file": history_file,
                    "summary_json": summary_json_string,
                    "initial_charts_json": chart_json_string,
                    "narratives_json": narrative_json_string,
                    "bad_dates_json": bad_dates_json,
                    "promo_issue_json": promo_issue_json,
                    "table_data_json": table_data_json_string
                }

                print(f"Rendering HTML template: {template_file}")
                success = render_html_template(context, template_file, output_html)

                if success:
                    print(f"--- Dashboard successfully generated: {output_html} ---")
                else:
                    print("--- Dashboard generation failed during HTML rendering. ---")
            else:
                # Report JSON validation failure
                print("\n--- ERROR: One or more JSON components failed validation. HTML rendering aborted. ---")
                print(f"Validation Results:\n"
                      f"  Summary JSON: {is_summary_valid}\n"
                      f"  Charts JSON: {is_chart_data_valid}\n"
                      f"  Narratives JSON: {is_narrative_valid}\n"
                      f"  Bad Dates JSON: {is_bad_dates_valid}\n"
                      f"  Promo Issues JSON: {is_promo_issue_valid}\n"
                      f"  Table Data JSON: {is_table_data_valid}")
                print("--- Dashboard generation failed. ---")
                # Optionally write a basic error HTML
                try:
                    # output_html is now accessible due to global declaration at function start
                    with open(output_html, 'w', encoding='utf-8') as f:
                         f.write("<html><body><h1>Dashboard Generation Failed</h1><p>Could not generate valid JSON data for all components. Please check script logs/output.</p></body></html>")
                    print(f"Basic error HTML written to {output_html}")
                except Exception as html_err:
                     print(f"Could not write error HTML file: {html_err}")

        except Exception as e:
            # Catch unexpected errors during component generation or rendering
            print(f"An unexpected error occurred during component generation or rendering: {e}")
            traceback.print_exc()
            print("--- Dashboard generation failed. ---")
    else:
        # Handle case where data loading failed
        print("No valid data loaded/processed. Dashboard generation aborted.")
        # Optionally write a basic error HTML
        try:
            # output_html is now accessible due to global declaration at function start
            with open(output_html, 'w', encoding='utf-8') as f:
                 f.write("<html><body><h1>Dashboard Generation Failed</h1><p>Could not load or process input data. Please check CSV files and script configuration.</p></body></html>")
            print(f"Basic error HTML written to {output_html}")
        except Exception as html_err:
             print(f"Could not write error HTML file: {html_err}")


# Inline Run

if __name__ == "__main__":
    main()

--- Starting Netflix Org Dashboard Generation ---
Report date: 2025-04-23
Loading data from employee_snapshot.csv and employee_history.csv...
Calculating organizational scope for managers...
Identifying data quality issues...
Found 61 potential date formatting issues.
Found 18 promotion timing issues (promo date before hire date).
Proceeding with 617 records having valid hire dates.
Generating dashboard components...
Generated 22 chart specifications.
Generated data for 6 tables.

--- Validating Generated JSON ---
--- JSON Validation Complete ---

Rendering HTML template: gem_ip_template.html
Successfully rendered template to gem_ip_dashboard.html
--- Dashboard successfully generated: gem_ip_dashboard.html ---
